In [ ]:
from imports import *
from FeatureExtraction import *
from PreProcessor import *
_PreProcessor = PreProcessor()


## Preprocessor

</h3>Train </h3>

In [2]:
UC_documents_train, UC_to_index_train,Vocab_UC_train= _PreProcessor.setupUC("./Dataset/teiid_dataset/train_UC", 'train')
code_documents_train, CC_to_index_train,Vocab_CC_train = _PreProcessor.setupCC("./Dataset/teiid_dataset/train_CC", 'train')

UC_documents_test, UC_to_index_test,Vocab_UC_test= _PreProcessor.setupUC("./Dataset/teiid_dataset/test_UC", 'test')
code_documents_test, CC_to_index_test,Vocab_CC_test = _PreProcessor.setupCC("./Dataset/teiid_dataset/test_CC", 'test')

Vocab_UC_train.update(Vocab_CC_train)

In [3]:
with open('./pickles/UC_documents_train.pkl', 'wb') as f:
       pickle.dump(UC_documents_train, f)
with open('./pickles/code_documents_train.pkl', 'wb') as f:
        pickle.dump(code_documents_train, f)
with open('./pickles/UCindex_train.pkl', 'wb') as f:
        pickle.dump(UC_to_index_train, f)
with open('./pickles/CCindex_train.pkl', 'wb') as f:
        pickle.dump(CC_to_index_train, f)
with open('./pickles/UCTokens_train.pkl', 'wb') as f:
        pickle.dump(Vocab_UC_train, f)
with open('./pickles/CodeTokens_train.pkl', 'wb') as f:
        pickle.dump(Vocab_CC_train, f)

</h3>Test</h3>

In [4]:
with open('./pickles/UC_documents_test.pkl', 'wb') as f:
       pickle.dump(UC_documents_test, f)
with open('./pickles/code_documents_test.pkl', 'wb') as f:
        pickle.dump(code_documents_test, f)
with open('./pickles/UCindex_test.pkl', 'wb') as f:
        pickle.dump(UC_to_index_test, f)
with open('./pickles/CCindex_test.pkl', 'wb') as f:
        pickle.dump(CC_to_index_test, f)
with open('./pickles/UCTokens_test.pkl', 'wb') as f:
        pickle.dump(Vocab_UC_test, f)
with open('./pickles/CodeTokens_test.pkl', 'wb') as f:
        pickle.dump(Vocab_CC_test, f)

In [5]:
UC_documents_train = np.load('./pickles/UC_documents_train.pkl',allow_pickle=True)
Vocab_UC_train = np.load('./pickles/UCTokens_train.pkl',allow_pickle=True)
code_documents_train = np.load('./pickles/code_documents_train.pkl',allow_pickle=True)
Vocab_CC_train = np.load('./pickles/CodeTokens_train.pkl',allow_pickle=True)
UC_to_index_train = np.load('./pickles/UCindex_train.pkl',allow_pickle=True)
CC_to_index_train = np.load('./pickles/CCindex_train.pkl',allow_pickle=True)


UC_documents_test = np.load('./pickles/UC_documents_test.pkl',allow_pickle=True)
Vocab_UC_test = np.load('./pickles/UCTokens_test.pkl',allow_pickle=True)
code_documents_test = np.load('./pickles/code_documents_test.pkl',allow_pickle=True)
Vocab_CC_test = np.load('./pickles/CodeTokens_test.pkl',allow_pickle=True)
UC_to_index_test = np.load('./pickles/UCindex_test.pkl',allow_pickle=True)
CC_to_index_test = np.load('./pickles/CCindex_test.pkl',allow_pickle=True)



Vocab_UC_train.update(Vocab_CC_train)

In [6]:
print(len(UC_documents_test))
print(len(code_documents_test))
print(len(UC_to_index_test))
print(len(CC_to_index_test))

print(len(UC_documents_train))
print(len(code_documents_train))
print(len(UC_to_index_train))
print(len(CC_to_index_train))

409
174
409
174
2398
1031
2398
1031


## Initialize Normalizer

In [8]:
normalizer = MinMaxScaler(copy=False)

## Adjuting the csv file

In [9]:
_PreProcessor.setupCSV("Dataset/teiid_dataset/train.csv", "Dataset/teiid_dataset/train_modified.csv",UC_to_index_train,CC_to_index_train)
_PreProcessor.setupCSV("Dataset/teiid_dataset/test.csv", "Dataset/teiid_dataset/test_modified.csv",UC_to_index_test,CC_to_index_test)

sum =  5043
sum =  207


## Collecting the 131 features

In [11]:
featureExtraction = FeatureExtraction(Vocab_UC_train)

# Feature extraction for training set
tfidf_matrix_uc_train, tfidf_matrix_code_train, idf_uc_dict_train, idf_code_dict_train, feature_names_uc_train, feature_names_code_train, df_uc_dict_train, df_code_dict_train = featureExtraction.TFIDFVectorizer(UC_documents_train, code_documents_train,train_or_test='train')
UC_count_matrix_train, code_count_matrix_train, tf_uc_dict_train, tf_code_dict_train = featureExtraction.CountVectorizerModel(UC_documents_train, code_documents_train, 'train')
idf_uc_train, idf_code_train = featureExtraction.IDFPreProcessing(UC_documents_train, idf_code_dict_train, code_documents_train, idf_uc_dict_train)
ictf_uc_train, ictf_code_train = featureExtraction.ICTFPreProcessing(UC_documents_train, tf_code_dict_train, code_documents_train, tf_uc_dict_train)

# Feature extraction for testing set
tfidf_matrix_uc_test, tfidf_matrix_code_test, idf_uc_dict_test, idf_code_dict_test, feature_names_uc_test, feature_names_code_test, df_uc_dict_test, df_code_dict_test = featureExtraction.TFIDFVectorizer(UC_documents_test, code_documents_test,train_or_test='test')
UC_count_matrix_test, code_count_matrix_test, tf_uc_dict_test, tf_code_dict_test = featureExtraction.CountVectorizerModel(UC_documents_test, code_documents_test, 'test')
idf_uc_test, idf_code_test = featureExtraction.IDFPreProcessing(UC_documents_test, idf_code_dict_test, code_documents_test, idf_uc_dict_test)
ictf_uc_test, ictf_code_test = featureExtraction.ICTFPreProcessing(UC_documents_test, tf_code_dict_test, code_documents_test, tf_uc_dict_test)
# # the values of the count matrices are normalized
#8.3 sec

In [ ]:
entropy_uc_train,entropy_code_train,variance_uc_train,variance_code_train=featureExtraction.EntropyPreProcessing(UC_documents_train,code_documents_train,idf_uc_dict_train,idf_code_dict_train,df_uc_dict_train,df_code_dict_train)
entropy_uc_test,entropy_code_test,variance_uc_test,variance_code_test=featureExtraction.EntropyPreProcessing(UC_documents_test,code_documents_test,idf_uc_dict_test,idf_code_dict_test,df_uc_dict_test,df_code_dict_test)
#224 min

In [ ]:
with open('./pickles/entropy_uc_train.pkl', 'wb') as f:
       pickle.dump(entropy_uc_train, f)
with open('./pickles/entropy_code_train.pkl', 'wb') as f:
        pickle.dump(entropy_code_train, f)
with open('./pickles/variance_uc_train.pkl', 'wb') as f:
        pickle.dump(variance_uc_train, f)
with open('./pickles/variance_code_train.pkl', 'wb') as f:
        pickle.dump(variance_code_train, f)

with open('./pickles/entropy_uc_test.pkl', 'wb') as f:
       pickle.dump(entropy_uc_test, f)
with open('./pickles/entropy_code_test.pkl', 'wb') as f:
        pickle.dump(entropy_code_test, f)
with open('./pickles/variance_uc_test.pkl', 'wb') as f:
        pickle.dump(variance_uc_test, f)
with open('./pickles/variance_code_test.pkl', 'wb') as f:
        pickle.dump(variance_code_test, f)

In [10]:
entropy_uc_train = np.load('./pickles/entropy_uc_train.pkl',allow_pickle=True)
entropy_code_train = np.load('./pickles/entropy_code_train.pkl',allow_pickle=True)
variance_uc_train = np.load('./pickles/variance_uc_train.pkl',allow_pickle=True)
variance_code_train = np.load('./pickles/variance_code_train.pkl',allow_pickle=True)
entropy_uc_test = np.load('./pickles/entropy_uc_test.pkl',allow_pickle=True)
entropy_code_test = np.load('./pickles/entropy_code_test.pkl',allow_pickle=True)
variance_uc_test = np.load('./pickles/variance_uc_test.pkl',allow_pickle=True)
variance_code_test = np.load('./pickles/variance_code_test.pkl',allow_pickle=True)

In [ ]:
# variance_uc_train,variance_code_train= featureExtraction.VarPreProcessing(UC_documents_train,code_documents_train,idf_uc_dict_train,idf_code_dict_train)
PMI_uc_train,PMI_code_train=featureExtraction.PMIPreProcessing(code_documents_train,UC_documents_train)
SCQ_uc_train,SCQ_code_train = featureExtraction.SCQPreProcessing(UC_documents_train,code_documents_train,tf_uc_dict_train,tf_code_dict_train,idf_uc_dict_train,idf_code_dict_train)

# variance_uc_test,variance_code_test= featureExtraction.VarPreProcessing(UC_documents_test,code_documents_test,idf_uc_dict_test,idf_code_dict_test)
PMI_uc_test,PMI_code_test=featureExtraction.PMIPreProcessing(code_documents_test,UC_documents_test)
SCQ_uc_test,SCQ_code_test = featureExtraction.SCQPreProcessing(UC_documents_test,code_documents_test,tf_uc_dict_test,tf_code_dict_test,idf_uc_dict_test,idf_code_dict_test)
#6min

In [ ]:
with open('./pickles/PMI_uc_train.pkl', 'wb') as f:
       pickle.dump(PMI_uc_train, f)
with open('./pickles/PMI_code_train.pkl', 'wb') as f:
        pickle.dump(PMI_code_train, f)

with open('./pickles/SCQ_uc_train.pkl', 'wb') as f:
         pickle.dump(SCQ_uc_train, f)
with open('./pickles/SCQ_code_train.pkl', 'wb') as f:
        pickle.dump(SCQ_code_train, f)

with open('./pickles/PMI_uc_test.pkl', 'wb') as f:
    pickle.dump(PMI_uc_test, f)

with open('./pickles/PMI_code_test.pkl', 'wb') as f:
    pickle.dump(PMI_code_test, f)

with open('./pickles/SCQ_uc_test.pkl', 'wb') as f:
    pickle.dump(SCQ_uc_test, f)

with open('./pickles/SCQ_code_test.pkl', 'wb') as f:
    pickle.dump(SCQ_code_test, f)

In [12]:
PMI_uc_train = np.load('./pickles/PMI_uc_train.pkl',allow_pickle=True)
PMI_code_train = np.load('./pickles/PMI_code_train.pkl',allow_pickle=True)
SCQ_uc_train = np.load('./pickles/SCQ_uc_train.pkl',allow_pickle=True)
SCQ_code_train = np.load('./pickles/SCQ_code_train.pkl',allow_pickle=True)
PMI_uc_test = np.load('./pickles/PMI_uc_test.pkl',allow_pickle=True)
PMI_code_test = np.load('./pickles/PMI_code_test.pkl',allow_pickle=True)
SCQ_uc_test = np.load('./pickles/SCQ_uc_test.pkl',allow_pickle=True)
SCQ_code_test = np.load('./pickles/SCQ_code_test.pkl',allow_pickle=True)

In [6]:
#-------------------------14 IR based features Train--------------------------#

# # 1) Vector space model
cosine_similarities_feature_train = featureExtraction.VectorSpaceModel(tfidf_matrix_uc_train, tfidf_matrix_code_train)
print("cosine_similarities_feature_train", cosine_similarities_feature_train.shape)

# # Our model will classify based on the ranks of the data rather than their actual values.
# cosine_similarity_UC_train = rankdata(-cosine_similarities_feature_train, method='dense', axis=1)
# print("cosine_similarity_UC_train", cosine_similarity_UC_train.shape)
# cosine_similarity_CC_train = rankdata(-cosine_similarities_feature_train, method='dense', axis=0)
# print("cosine_similarity_CC_train", cosine_similarity_CC_train.shape)


# np.save('./pickles/cosine_similarity_UC_train.npy', cosine_similarity_UC_train)
# np.save('./pickles/cosine_similarity_CC_train.npy', cosine_similarity_CC_train)

# # 2) Latent semantic analysis
# LSA_similarities_feature_train = featureExtraction.LSA(tfidf_matrix_uc_train, tfidf_matrix_code_train)
# print('LSA similarity', LSA_similarities_feature_train.shape)

# LSA_similarities_UC_train = rankdata(-LSA_similarities_feature_train, method='dense', axis=1)
# print("LSA_similarities_UC_train", LSA_similarities_UC_train.shape)
# LSA_similarities_CC_train = rankdata(-LSA_similarities_feature_train, method='dense', axis=0)
# print("LSA_similarities_CC_train", LSA_similarities_CC_train.shape)


# np.save('./pickles/LSA_similarities_UC_train.npy', LSA_similarities_UC_train)
# np.save('./pickles/LSA_similarities_CC_train.npy', LSA_similarities_CC_train)

# 3) Latent Dirichlet Allocation
# DocumentTopicDisUC_dense_train, DocumentTopicDisCode_dense_train, cosine_similarities_LDA_train = featureExtraction.LDA(UC_documents_train, code_documents_train, Vocab_UC_train)
# print('LDA similarity', cosine_similarities_LDA_train.shape)

# LDA_similarities_UC_train = rankdata(-cosine_similarities_LDA_train, method='dense', axis=1)
# print("LDA_similarities_UC_train", LDA_similarities_UC_train.shape)
# LDA_similarities_CC_train = rankdata(-cosine_similarities_LDA_train, 'dense', axis=0)
# print("LDA_similarities_CC_train", LDA_similarities_CC_train.shape)

# np.save('./pickles/LDA_similarities_UC_train.npy', LDA_similarities_UC_train)
# np.save('./pickles/LDA_similarities_CC_train.npy', LDA_similarities_CC_train)

# 4) Jensen-Shannon(JS)
# JS_features_train = featureExtraction.JensenShannon(UC_count_matrix_train, code_count_matrix_train)
# print('JS', JS_features_train.shape)

# JS_UC_train = rankdata(-JS_features_train, method='dense', axis=1)
# print('JS_UC_train', JS_UC_train.shape)
# JS_CC_train = rankdata(-JS_features_train, method='dense', axis=0)
# print('JS_CC_train', JS_CC_train.shape)

# np.save('./pickles/JS_UC_train.npy', JS_UC_train)
# np.save('./pickles/JS_CC_train.npy', JS_CC_train)

# 5)  Okapi BM25
# UC means query is UC and document is code
print("BM25_UC_train", len(UC_documents_train), len(code_documents_train), code_count_matrix_train.shape)
BM25_UC_train = featureExtraction.BM25(UC_documents_train, code_documents_train, idf_code_dict_train, code_count_matrix_train)
BM25_CC_train = featureExtraction.BM25(code_documents_train, UC_documents_train, idf_uc_dict_train, UC_count_matrix_train)

BM25_UC_train = rankdata(-BM25_UC_train, method='dense', axis=0)
print("BM25_UC_train", BM25_UC_train.shape)
BM25_CC_train = rankdata(-BM25_CC_train, method='dense', axis=0)
print("BM25_CC_train", BM25_CC_train.shape)

np.save('./pickles/BM25_UC_train.npy', BM25_UC_train)
np.save('./pickles/BM25_CC_train.npy', BM25_CC_train)

# 6) Language Model with Dirichlet
# UC means UC is query and code is doc
JM_UC_train = featureExtraction.SmoothingMethods(UC_documents_train, code_documents_train, code_count_matrix_train, tf_code_dict_train, JM_or_DP=True)
JM_CC_train = featureExtraction.SmoothingMethods(code_documents_train, UC_documents_train, UC_count_matrix_train, tf_uc_dict_train, True)

DP_UC_train = featureExtraction.SmoothingMethods(UC_documents_train, code_documents_train, code_count_matrix_train, tf_code_dict_train, JM_or_DP=False)
DP_CC_train = featureExtraction.SmoothingMethods(code_documents_train, UC_documents_train, UC_count_matrix_train, tf_uc_dict_train, False)

JM_UC_train = rankdata(-JM_UC_train, method='dense', axis=0)
print("JM_UC_train", JM_UC_train.shape)
JM_CC_train = rankdata(-JM_CC_train, method='dense', axis=0)
print("JM_CC_train", JM_CC_train.shape)
DP_UC_train = rankdata(-DP_UC_train, method='dense', axis=0)
print("DP_UC_train", DP_UC_train.shape)
DP_CC_train = rankdata(-DP_CC_train, method='dense', axis=0)
print("DP_CC_train", DP_CC_train.shape)


np.save('./pickles/JM_UC_train.npy', JM_UC_train)
np.save('./pickles/JM_CC_train.npy', JM_CC_train)


np.save('./pickles/DP_UC_train.npy', DP_UC_train)
np.save('./pickles/DP_CC_train.npy', DP_CC_train)


#-------------------------14 IR based features Test--------------------------#

# 1) Vector space model
cosine_similarities_feature_test = featureExtraction.VectorSpaceModel(tfidf_matrix_uc_test, tfidf_matrix_code_test)
print("cosine_similarities_feature_test", cosine_similarities_feature_test.shape)

# Our model will classify based on the ranks of the data rather than their actual values.
cosine_similarity_UC_test = rankdata(-cosine_similarities_feature_test, method='dense', axis=1)
print("cosine_similarity_UC_test", cosine_similarity_UC_test.shape)
cosine_similarity_CC_test = rankdata(-cosine_similarities_feature_test, method='dense', axis=0)
print("cosine_similarity_CC_test", cosine_similarity_CC_test.shape)


np.save('./pickles/cosine_similarity_UC_test.npy', cosine_similarity_UC_test)
np.save('./pickles/cosine_similarity_CC_test.npy', cosine_similarity_CC_test)

# 2) Latent semantic analysis
LSA_similarities_feature_test = featureExtraction.LSA(tfidf_matrix_uc_test, tfidf_matrix_code_test)
print('LSA similarity', LSA_similarities_feature_test.shape)

LSA_similarities_UC_test = rankdata(-LSA_similarities_feature_test, method='dense', axis=1)
print("LSA_similarities_UC_test", LSA_similarities_UC_test.shape)
LSA_similarities_CC_test = rankdata(-LSA_similarities_feature_test, method='dense', axis=0)
print("LSA_similarities_CC_test", LSA_similarities_CC_test.shape)

np.save('./pickles/LSA_similarities_UC_test.npy', LSA_similarities_UC_test)
np.save('./pickles/LSA_similarities_CC_test.npy', LSA_similarities_CC_test)

# 3) Latent Dirichlet Allocation
DocumentTopicDisUC_dense_test, DocumentTopicDisCode_dense_test, cosine_similarities_LDA_test = featureExtraction.LDA(UC_documents_test, code_documents_test, Vocab_UC_test)
print('LDA similarity', cosine_similarities_LDA_test.shape)

LDA_similarities_UC_test = rankdata(-cosine_similarities_LDA_test, method='dense', axis=1)
print("LDA_similarities_UC_test", LDA_similarities_UC_test.shape)
LDA_similarities_CC_test = rankdata(-cosine_similarities_LDA_test, 'dense', axis=0)
print("LDA_similarities_CC_test", LDA_similarities_CC_test.shape)

np.save('./pickles/LDA_similarities_UC_test.npy', LDA_similarities_UC_test)
np.save('./pickles/LDA_similarities_CC_test.npy', LDA_similarities_CC_test)

# 4) Jensen-Shannon(JS)
JS_features_test = featureExtraction.JensenShannon(UC_count_matrix_test, code_count_matrix_test)
print('JS', JS_features_test.shape)

JS_UC_test = rankdata(-JS_features_test, method='dense', axis=1)
print('JS_UC_test', JS_UC_test.shape)
JS_CC_test = rankdata(-JS_features_test, method='dense', axis=0)
print('JS_CC_test', JS_CC_test.shape)

np.save('./pickles/JS_UC_test.npy', JS_UC_test)
np.save('./pickles/JS_CC_test.npy', JS_CC_test)

# 5)  Okapi BM25
# UC means query is UC and document is code
BM25_UC_test = featureExtraction.BM25(UC_documents_test, code_documents_test, idf_code_dict_test, code_count_matrix_test)
BM25_CC_test = featureExtraction.BM25(code_documents_test, UC_documents_test, idf_uc_dict_test, UC_count_matrix_test)

BM25_UC_test = rankdata(-BM25_UC_test, method='dense', axis=0)
print("BM25_UC_test", BM25_UC_test.shape)
BM25_CC_test = rankdata(-BM25_CC_test, method='dense', axis=0)
print("BM25_CC_test", BM25_CC_test.shape)

np.sa ve('./pickles/BM25_UC_test.npy', BM25_UC_test)
np.save('./pickles/BM25_CC_test.npy', BM25_CC_test)

# 6) Language Model with Dirichlet
# UC means UC is query and code is doc
JM_UC_test = featureExtraction.SmoothingMethods(UC_documents_test, code_documents_test, code_count_matrix_test, tf_code_dict_test, JM_or_DP=True)
JM_CC_test = featureExtraction.SmoothingMethods(code_documents_test, UC_documents_test, UC_count_matrix_test, tf_uc_dict_test, True)

DP_UC_test = featureExtraction.SmoothingMethods(UC_documents_test, code_documents_test, code_count_matrix_test, tf_code_dict_test, JM_or_DP=False)
DP_CC_test = featureExtraction.SmoothingMethods(code_documents_test, UC_documents_test, UC_count_matrix_test, tf_uc_dict_test, False)

JM_UC_test = rankdata(-JM_UC_test, method='dense', axis=0)
print("JM_UC_test", JM_UC_test.shape)
JM_CC_test = rankdata(-JM_CC_test, method='dense', axis=0)
print("JM_CC_test", JM_CC_test.shape)
DP_UC_test = rankdata(-DP_UC_test, method='dense', axis=0)
print("DP_UC_test", DP_UC_test.shape)
DP_CC_test = rankdata(-DP_CC_test, method='dense', axis=0)
print("DP_CC_test", DP_CC_test.shape)

np.save('./pickles/JM_UC_test.npy', JM_UC_test)
np.save('./pickles/JM_CC_test.npy', JM_CC_test)

np.save('./pickles/DP_UC_test.npy', DP_UC_test)
np.save('./pickles/DP_CC_test.npy', DP_CC_test)


#200 MIN

JS (2398, 1031)
JS_UC_train (2398, 1031)
JS_CC_train (2398, 1031)
BM25_UC_train (1031, 2398)
BM25_CC_train (2398, 1031)
JM_UC_train (1031, 2398)
JM_CC_train (2398, 1031)
DP_UC_train (1031, 2398)
DP_CC_train (2398, 1031)
cosine_similarities_feature_test (409, 174)
cosine_similarity_UC_test (409, 174)
cosine_similarity_CC_test (409, 174)
LSA similarity (409, 174)
LSA_similarities_UC_test (409, 174)
LSA_similarities_CC_test (409, 174)
LDA similarity (409, 174)
LDA_similarities_UC_test (409, 174)
LDA_similarities_CC_test (409, 174)
JS (409, 174)
JS_UC_test (409, 174)
JS_CC_test (409, 174)
BM25_UC_test (174, 409)
BM25_CC_test (409, 174)
JM_UC_test (174, 409)
JM_CC_test (409, 174)
DP_UC_test (174, 409)
DP_CC_test (409, 174)


In [13]:
#-----------------------IR Train Features Loading------------------#
cosine_similarity_UC_train = np.load('./pickles/cosine_similarity_UC_train.npy')
cosine_similarity_UC_train = normalizer.fit_transform(cosine_similarity_UC_train)

cosine_similarity_CC_train = np.load('./pickles/cosine_similarity_CC_train.npy')
cosine_similarity_CC_train = normalizer.fit_transform(cosine_similarity_CC_train)

LSA_similarities_UC_train = np.load('./pickles/LSA_similarities_UC_train.npy')
LSA_similarities_UC_train = normalizer.fit_transform(LSA_similarities_UC_train)

LSA_similarities_CC_train = np.load('./pickles/LSA_similarities_CC_train.npy')
LSA_similarities_CC_train = normalizer.fit_transform(LSA_similarities_CC_train)

LDA_similarities_UC_train = np.load('./pickles/LDA_similarities_UC_train.npy')
LDA_similarities_UC_train = normalizer.fit_transform(LDA_similarities_UC_train)

LDA_similarities_CC_train = np.load('./pickles/LDA_similarities_CC_train.npy')
LDA_similarities_CC_train = normalizer.fit_transform(LDA_similarities_CC_train)

JS_UC_train = np.load('./pickles/JS_UC_train.npy')
JS_UC_train = normalizer.fit_transform(JS_UC_train)

JS_CC_train = np.load('./pickles/JS_CC_train.npy')
JS_CC_train = normalizer.fit_transform(JS_CC_train)

BM25_UC_train = np.load('./pickles/BM25_UC_train.npy')
BM25_UC_train = normalizer.fit_transform(BM25_UC_train)

BM25_CC_train = np.load('./pickles/BM25_CC_train.npy')
BM25_CC_train = normalizer.fit_transform(BM25_CC_train)

JM_UC_train = np.load('./pickles/JM_UC_train.npy')
JM_UC_train = normalizer.fit_transform(JM_UC_train)

JM_CC_train = np.load('./pickles/JM_CC_train.npy')
JM_CC_train = normalizer.fit_transform(JM_CC_train)

DP_UC_train = np.load('./pickles/DP_UC_train.npy')
DP_UC_train = normalizer.fit_transform(DP_UC_train)

DP_CC_train = np.load('./pickles/DP_CC_train.npy')
DP_CC_train = normalizer.fit_transform(DP_CC_train)


In [14]:
#-----------------------IR Test Features Loading------------------#
cosine_similarity_UC_test = np.load('./pickles/cosine_similarity_UC_test.npy')
cosine_similarity_UC_test = normalizer.fit_transform(cosine_similarity_UC_test)

cosine_similarity_CC_test = np.load('./pickles/cosine_similarity_CC_test.npy')
cosine_similarity_CC_test = normalizer.fit_transform(cosine_similarity_CC_test)

LSA_similarities_UC_test = np.load('./pickles/LSA_similarities_UC_test.npy')
LSA_similarities_UC_test = normalizer.fit_transform(LSA_similarities_UC_test)

LSA_similarities_CC_test = np.load('./pickles/LSA_similarities_CC_test.npy')
LSA_similarities_CC_test = normalizer.fit_transform(LSA_similarities_CC_test)

LDA_similarities_UC_test = np.load('./pickles/LDA_similarities_UC_test.npy')
LDA_similarities_UC_test = normalizer.fit_transform(LDA_similarities_UC_test)

LDA_similarities_CC_test = np.load('./pickles/LDA_similarities_CC_test.npy')
LDA_similarities_CC_test = normalizer.fit_transform(LDA_similarities_CC_test)

JS_UC_test = np.load('./pickles/JS_UC_test.npy')
JS_UC_test = normalizer.fit_transform(JS_UC_test)

JS_CC_test = np.load('./pickles/JS_CC_test.npy')
JS_CC_test = normalizer.fit_transform(JS_CC_test)

BM25_UC_test = np.load('./pickles/BM25_UC_test.npy')
BM25_UC_test = normalizer.fit_transform(BM25_UC_test)

BM25_CC_test = np.load('./pickles/BM25_CC_test.npy')
BM25_CC_test = normalizer.fit_transform(BM25_CC_test)

JM_UC_test = np.load('./pickles/JM_UC_test.npy')
JM_UC_test = normalizer.fit_transform(JM_UC_test)

JM_CC_test = np.load('./pickles/JM_CC_test.npy')
JM_CC_test = normalizer.fit_transform(JM_CC_test)

DP_UC_test = np.load('./pickles/DP_UC_test.npy')
DP_UC_test = normalizer.fit_transform(DP_UC_test)

DP_CC_test = np.load('./pickles/DP_CC_test.npy')
DP_CC_test = normalizer.fit_transform(DP_CC_test)

In [15]:
# ------------------------pre-retrieval (21 metrics) Train--------------------------#
# 1) IDF Features
avg_idf_uc_train = featureExtraction.AvgIDF(idf_uc_train)
avg_idf_code_train = featureExtraction.AvgIDF(idf_code_train)

print('avg_idf_uc_train_shape', avg_idf_uc_train.shape) 
print('avg_idf_code_train_shape', avg_idf_code_train.shape) 

max_idf_uc_train = featureExtraction.MaxIDF(idf_uc_train).reshape(1,-1) 
max_idf_code_train = featureExtraction.MaxIDF(idf_code_train).reshape(1,-1) 

print('max_idf_uc_train_shape', max_idf_uc_train.shape) 
print('max_idf_code_train_shape', max_idf_code_train.shape) 

dev_idf_uc_train = featureExtraction.DevIDF(idf_uc_train).reshape(1,-1) 
dev_idf_code_train = featureExtraction.DevIDF(idf_code_train).reshape(1,-1) 

print('dev_idf_uc_train_shape', dev_idf_uc_train.shape) 
print('dev_idf_code_train_shape', dev_idf_code_train.shape) 

# 2) ICTF Features
avg_ictf_uc_train = featureExtraction.AvgICTF(ictf_uc_train).reshape(1,-1) 
avg_ictf_code_train = featureExtraction.AvgICTF(ictf_code_train).reshape(1,-1) 

print('avg_ictf_uc_train_shape', avg_ictf_uc_train.shape)
print('avg_ictf_code_train_shape', avg_ictf_code_train.shape)

max_ictf_uc_train = featureExtraction.MaxICTF(ictf_uc_train).reshape(1,-1) 
max_ictf_code_train = featureExtraction.MaxICTF(ictf_code_train).reshape(1,-1) 

print('max_ictf_uc_train_shape', max_ictf_uc_train.shape)
print('max_ictf_code_train_shape', max_ictf_code_train.shape)

dev_ictf_uc_train = featureExtraction.DevICTF(ictf_uc_train).reshape(1,-1) 
dev_ictf_code_train = featureExtraction.DevICTF(ictf_code_train).reshape(1,-1) 

print('dev_ictf_uc_train_shape', dev_ictf_uc_train.shape)
print('dev_ictf_code_train_shape', dev_ictf_code_train.shape)
# 3) Entropy Features

avg_entropy_uc_train = featureExtraction.AvgEntropy(entropy_uc_train).reshape(1,-1) 
avg_entropy_code_train = featureExtraction.AvgEntropy(entropy_code_train).reshape(1,-1) 

print('avg_entropy_uc_train_shape', avg_entropy_uc_train.shape)
print('avg_entropy_code_train_shape', avg_entropy_code_train.shape)

max_entropy_uc_train = featureExtraction.MaxEntropy(entropy_uc_train).reshape(1,-1) 
max_entropy_code_train = featureExtraction.MaxEntropy(entropy_code_train).reshape(1,-1) 

print('max_entropy_uc_train_shape', max_entropy_uc_train.shape)
print('max_entropy_code_train_shape', max_entropy_code_train.shape)

med_entropy_uc_train = featureExtraction.MedEntropy(entropy_uc_train).reshape(1,-1) 
med_entropy_code_train = featureExtraction.MedEntropy(entropy_code_train).reshape(1,-1) 

print('med_entropy_uc_train_shape', med_entropy_uc_train.shape)
print('med_entropy_code_train_shape', med_entropy_code_train.shape)

dev_entropy_uc_train = featureExtraction.DevEntropy(entropy_uc_train).reshape(1,-1) 
dev_entropy_code_train = featureExtraction.DevEntropy(entropy_code_train).reshape(1,-1) 

print('dev_entropy_uc_train_shape', dev_entropy_uc_train.shape)
print('dev_entropy_code_train_shape', dev_entropy_code_train.shape)
      

# 4) Variance Features

avg_variance_uc_train = featureExtraction.AvgVariance(variance_uc_train).reshape(1,-1) 
avg_variance_code_train = featureExtraction.AvgVariance(variance_code_train).reshape(1,-1) 

print('avg_variance_uc_train_shape', avg_variance_uc_train.shape)
print('avg_variance_code_train_shape', avg_variance_code_train.shape)

max_variance_uc_train = featureExtraction.MaxVariance(variance_uc_train).reshape(1,-1) 
max_variance_code_train = featureExtraction.MaxVariance(variance_code_train).reshape(1,-1) 

print('max_variance_uc_train_shape', max_variance_uc_train.shape)
print('max_variance_code_train_shape', max_variance_code_train.shape)

sum_variance_uc_train = featureExtraction.SumVariance(variance_uc_train).reshape(1,-1) 
sum_variance_code_train = featureExtraction.SumVariance(variance_code_train).reshape(1,-1) 

print('sum_variance_uc_train_shape', sum_variance_uc_train.shape)
print('sum_variance_code_train_shape', sum_variance_code_train.shape)
# 5) SCQ Features
avg_scq_uc_train = featureExtraction.AvgSCQ(SCQ_uc_train).reshape(1,-1) 
avg_scq_code_train = featureExtraction.AvgSCQ(SCQ_code_train).reshape(1,-1) 

print('avg_scq_uc_train_shape', avg_scq_uc_train.shape)
print('avg_scq_code_train_shape', avg_scq_code_train.shape)

max_scq_uc_train = featureExtraction.MaxSCQ(SCQ_uc_train).reshape(1,-1) 
max_scq_code_train = featureExtraction.MaxSCQ(SCQ_code_train).reshape(1,-1) 

print('max_scq_uc_train_shape', max_scq_uc_train.shape)
print('max_scq_code_train_shape', max_scq_code_train.shape)

sum_sqc_uc_train = featureExtraction.SumSCQ(SCQ_uc_train).reshape(1,-1) 
sum_sqc_code_train = featureExtraction.SumSCQ(SCQ_code_train).reshape(1,-1) 

print('sum_sqc_uc_train_shape', sum_sqc_uc_train.shape)
print('sum_sqc_code_train_shape', sum_sqc_code_train.shape)

# 6) PMI Features

avg_pmi_uc_train = featureExtraction.AvgPMI(PMI_uc_train).reshape(1,-1) 
avg_pmi_code_train = featureExtraction.AvgPMI(PMI_code_train).reshape(1,-1) 

print('avg_pmi_uc_train_shape', avg_pmi_uc_train.shape)
print('avg_pmi_code_train_shape', avg_pmi_code_train.shape)

max_pmi_uc_train = featureExtraction.MaxPMI(PMI_uc_train).reshape(1,-1) 
max_pmi_code_train = featureExtraction.MaxPMI(PMI_code_train).reshape(1,-1) 

print('max_pmi_uc_train_shape', max_pmi_uc_train.shape)
print('max_pmi_code_train_shape', max_pmi_code_train.shape)
# 7) QS Features

qs_uc_train = featureExtraction.QS(UC_documents_train, code_documents_train).reshape(1,-1) 
qs_code_train = featureExtraction.QS(code_documents_train, UC_documents_train).reshape(1,-1) 

print('qs_uc_train_shape', qs_uc_train.shape)
print('qs_code_train_shape', qs_code_train.shape)

# 8) SCS Features
UC_SCS_train = featureExtraction.simplifiedClarityScore(UC_documents_train, UC_count_matrix_train, tf_code_dict_train).reshape(1,-1) 
CC_SCS_train = featureExtraction.simplifiedClarityScore(code_documents_train, code_count_matrix_train, tf_uc_dict_train).reshape(1,-1) 
print(tf_code_dict_train)

print('UC_SCS_train_shape', UC_SCS_train.shape)
print('CC_SCS_train_shape', CC_SCS_train.shape)

# 9) Coherence Score Features
UC_CoherenceScore_train = featureExtraction.CoherenceScore(UC_documents_train, tfidf_matrix_code_train).reshape(1,-1) 
CC_CoherenceScore_train = featureExtraction.CoherenceScore(code_documents_train, tfidf_matrix_uc_train).reshape(1,-1) 

print('UC_CoherenceScore_train_shape', UC_CoherenceScore_train.shape)
print('CC_CoherenceScore_train_shape', CC_CoherenceScore_train.shape)

#75min

avg_idf_uc_train_shape (2398,)
avg_idf_code_train_shape (1031,)
max_idf_uc_train_shape (1, 2398)
max_idf_code_train_shape (1, 1031)
dev_idf_uc_train_shape (1, 2398)
dev_idf_code_train_shape (1, 1031)
avg_ictf_uc_train_shape (1, 2398)
avg_ictf_code_train_shape (1, 1031)
max_ictf_uc_train_shape (1, 2398)
max_ictf_code_train_shape (1, 1031)
dev_ictf_uc_train_shape (1, 2398)
dev_ictf_code_train_shape (1, 1031)
avg_entropy_uc_train_shape (1, 2398)
avg_entropy_code_train_shape (1, 1031)
max_entropy_uc_train_shape (1, 2398)
max_entropy_code_train_shape (1, 1031)
med_entropy_uc_train_shape (1, 2398)
med_entropy_code_train_shape (1, 1031)
dev_entropy_uc_train_shape (1, 2398)
dev_entropy_code_train_shape (1, 1031)
avg_variance_uc_train_shape (1, 2398)
avg_variance_code_train_shape (1, 1031)
max_variance_uc_train_shape (1, 2398)
max_variance_code_train_shape (1, 1031)
sum_variance_uc_train_shape (1, 2398)
sum_variance_code_train_shape (1, 1031)
avg_scq_uc_train_shape (1, 2398)
avg_scq_code_train_

KeyboardInterrupt: 

In [ ]:
#----------------saving pre retrival Train------------------#
np.save('./pickles/avg_idf_uc_train.npy', avg_idf_uc_train)
np.save('./pickles/avg_idf_code_train.npy', avg_idf_code_train)

np.save('./pickles/max_idf_uc_train.npy', max_idf_uc_train)
np.save('./pickles/max_idf_code_train.npy', max_idf_code_train)

np.save('./pickles/dev_idf_uc_train.npy', dev_idf_uc_train)
np.save('./pickles/dev_idf_code_train.npy', dev_idf_code_train)

np.save('./pickles/avg_ictf_uc_train.npy', avg_ictf_uc_train)
np.save('./pickles/avg_ictf_code_train.npy', avg_ictf_code_train)

np.save('./pickles/max_ictf_uc_train.npy', max_ictf_uc_train)
np.save('./pickles/max_ictf_code_train.npy', max_ictf_code_train)

np.save('./pickles/dev_ictf_uc_train.npy', dev_ictf_uc_train)
np.save('./pickles/dev_ictf_code_train.npy', dev_ictf_code_train)

np.save('./pickles/avg_entropy_uc_train.npy', avg_entropy_uc_train)
np.save('./pickles/avg_entropy_code_train.npy', avg_entropy_code_train)

np.save('./pickles/max_entropy_uc_train.npy', max_entropy_uc_train)
np.save('./pickles/max_entropy_code_train.npy', max_entropy_code_train)

np.save('./pickles/max_entropy_uc_train.npy', max_entropy_uc_train)
np.save('./pickles/max_entropy_code_train.npy', max_entropy_code_train)

np.save('./pickles/med_entropy_uc_train.npy', med_entropy_uc_train)
np.save('./pickles/med_entropy_code_train.npy', med_entropy_code_train)

np.save('./pickles/dev_entropy_uc_train.npy', dev_entropy_uc_train)
np.save('./pickles/dev_entropy_code_train.npy', dev_entropy_code_train)

np.save('./pickles/avg_variance_uc_train.npy', avg_variance_uc_train)
np.save('./pickles/avg_variance_code_train.npy', avg_variance_code_train)

np.save('./pickles/max_variance_uc_train.npy', max_variance_uc_train)
np.save('./pickles/max_variance_code_train.npy', max_variance_code_train)

np.save('./pickles/sum_variance_uc_train.npy', sum_variance_uc_train)
np.save('./pickles/sum_variance_code_train.npy', sum_variance_code_train)

np.save('./pickles/avg_scq_uc_train.npy', avg_scq_uc_train)
np.save('./pickles/avg_scq_code_train.npy', avg_scq_code_train)

np.save('./pickles/max_scq_uc_train.npy', max_scq_uc_train)
np.save('./pickles/max_scq_code_train.npy', max_scq_code_train)

np.save('./pickles/sum_sqc_uc_train.npy', sum_sqc_uc_train)
np.save('./pickles/sum_sqc_code_train.npy', sum_sqc_code_train)

np.save('./pickles/avg_pmi_uc_train.npy', avg_pmi_uc_train)
np.save('./pickles/avg_pmi_code_train.npy', avg_pmi_code_train)

np.save('./pickles/max_pmi_uc_train.npy', max_pmi_uc_train)
np.save('./pickles/max_pmi_code_train.npy', max_pmi_code_train)

np.save('./pickles/qs_uc_train.npy', qs_uc_train)
np.save('./pickles/qs_code_train.npy', qs_code_train)

np.save('./pickles/UC_SCS_train.npy', UC_SCS_train)
np.save('./pickles/CC_SCS_train.npy', CC_SCS_train)

np.save('./pickles/UC_CoherenceScore_train.npy', UC_CoherenceScore_train)
np.save('./pickles/CC_CoherenceScore_train.npy', CC_CoherenceScore_train)

In [ ]:
# ------------------------pre-retrieval (21 metrics) Test--------------------------#
# 1) IDF Features
avg_idf_uc_test = featureExtraction.AvgIDF(idf_uc_test).reshape(1,-1)
avg_idf_code_test = featureExtraction.AvgIDF(idf_code_test).reshape(1,-1)

print('avg_idf_uc_test_shape', avg_idf_uc_test.shape)
print('avg_idf_code_test_shape', avg_idf_code_test.shape)

max_idf_uc_test = featureExtraction.MaxIDF(idf_uc_test).reshape(1,-1)
max_idf_code_test = featureExtraction.MaxIDF(idf_code_test).reshape(1,-1)

print('max_idf_uc_test_shape', max_idf_uc_test.shape)
print('max_idf_code_test_shape', max_idf_code_test.shape)

dev_idf_uc_test = featureExtraction.DevIDF(idf_uc_test).reshape(1,-1)
dev_idf_code_test = featureExtraction.DevIDF(idf_code_test).reshape(1,-1)

print('dev_idf_uc_test_shape', dev_idf_uc_test.shape)
print('dev_idf_code_test_shape', dev_idf_code_test.shape)

# 2) ICTF Features
avg_ictf_uc_test = featureExtraction.AvgICTF(ictf_uc_test).reshape(1,-1)
avg_ictf_code_test = featureExtraction.AvgICTF(ictf_code_test).reshape(1,-1)

print('avg_ictf_uc_test_shape', avg_ictf_uc_test.shape)
print('avg_ictf_code_test_shape', avg_ictf_code_test.shape)

max_ictf_uc_test = featureExtraction.MaxICTF(ictf_uc_test).reshape(1,-1)
max_ictf_code_test = featureExtraction.MaxICTF(ictf_code_test).reshape(1,-1)

print('max_ictf_uc_test_shape', max_ictf_uc_test.shape)
print('max_ictf_code_test_shape', max_ictf_code_test.shape)

dev_ictf_uc_test = featureExtraction.DevICTF(ictf_uc_test).reshape(1,-1)
dev_ictf_code_test = featureExtraction.DevICTF(ictf_code_test).reshape(1,-1)

print('dev_ictf_uc_test_shape', dev_ictf_uc_test.shape)
print('dev_ictf_code_test_shape', dev_ictf_code_test.shape)

# 3) Entropy Features
avg_entropy_uc_test = featureExtraction.AvgEntropy(entropy_uc_test).reshape(1,-1)
avg_entropy_code_test = featureExtraction.AvgEntropy(entropy_code_test).reshape(1,-1)

print('avg_entropy_uc_test_shape', avg_entropy_uc_test.shape)
print('avg_entropy_code_test_shape', avg_entropy_code_test.shape)

max_entropy_uc_test = featureExtraction.MaxEntropy(entropy_uc_test).reshape(1,-1)
max_entropy_code_test = featureExtraction.MaxEntropy(entropy_code_test).reshape(1,-1)

print('max_entropy_uc_test_shape', max_entropy_uc_test.shape)
print('max_entropy_code_test_shape', max_entropy_code_test.shape)

med_entropy_uc_test = featureExtraction.MedEntropy(entropy_uc_test).reshape(1,-1)
med_entropy_code_test = featureExtraction.MedEntropy(entropy_code_test).reshape(1,-1)

print('med_entropy_uc_test_shape', med_entropy_uc_test.shape)
print('med_entropy_code_test_shape', med_entropy_code_test.shape)

dev_entropy_uc_test = featureExtraction.DevEntropy(entropy_uc_test).reshape(1,-1)
dev_entropy_code_test = featureExtraction.DevEntropy(entropy_code_test).reshape(1,-1)

print('dev_entropy_uc_test_shape', dev_entropy_uc_test.shape)
print('dev_entropy_code_test_shape', dev_entropy_code_test.shape)

# 4) Variance Features

avg_variance_uc_test = featureExtraction.AvgVariance(variance_uc_test).reshape(1,-1) 
avg_variance_code_test = featureExtraction.AvgVariance(variance_code_test).reshape(1,-1) 

print('avg_variance_uc_test_shape', avg_variance_uc_test.shape)
print('avg_variance_code_test_shape', avg_variance_code_test.shape)

max_variance_uc_test = featureExtraction.MaxVariance(variance_uc_test).reshape(1,-1) 
max_variance_code_test = featureExtraction.MaxVariance(variance_code_test).reshape(1,-1) 

print('max_variance_uc_test_shape', max_variance_uc_test.shape)
print('max_variance_code_test_shape', max_variance_code_test.shape)

sum_variance_uc_test = featureExtraction.SumVariance(variance_uc_test).reshape(1,-1) 
sum_variance_code_test = featureExtraction.SumVariance(variance_code_test).reshape(1,-1) 

print('sum_variance_uc_test_shape', sum_variance_uc_test.shape)
print('sum_variance_code_test_shape', sum_variance_code_test.shape)

# 5) SCQ Features
avg_scq_uc_test = featureExtraction.AvgSCQ(SCQ_uc_test).reshape(1,-1) 
avg_scq_code_test = featureExtraction.AvgSCQ(SCQ_code_test).reshape(1,-1) 

print('avg_scq_uc_test_shape', avg_scq_uc_test.shape)
print('avg_scq_code_test_shape', avg_scq_code_test.shape)

max_scq_uc_test = featureExtraction.MaxSCQ(SCQ_uc_test).reshape(1,-1) 
max_scq_code_test = featureExtraction.MaxSCQ(SCQ_code_test).reshape(1,-1) 

print('max_scq_uc_test_shape', max_scq_uc_test.shape)
print('max_scq_code_test_shape', max_scq_code_test.shape)

sum_sqc_uc_test = featureExtraction.SumSCQ(SCQ_uc_test).reshape(1,-1) 
sum_sqc_code_test = featureExtraction.SumSCQ(SCQ_code_test).reshape(1,-1) 

print('sum_sqc_uc_test_shape', sum_sqc_uc_test.shape)
print('sum_sqc_code_test_shape', sum_sqc_code_test.shape)

# 6) PMI Features

avg_pmi_uc_test = featureExtraction.AvgPMI(PMI_uc_test).reshape(1,-1) 
avg_pmi_code_test = featureExtraction.AvgPMI(PMI_code_test).reshape(1,-1) 

print('avg_pmi_uc_test_shape', avg_pmi_uc_test.shape)
print('avg_pmi_code_test_shape', avg_pmi_code_test.shape)

max_pmi_uc_test = featureExtraction.MaxPMI(PMI_uc_test).reshape(1,-1) 
max_pmi_code_test = featureExtraction.MaxPMI(PMI_code_test).reshape(1,-1) 

print('max_pmi_uc_test_shape', max_pmi_uc_test.shape)
print('max_pmi_code_test_shape', max_pmi_code_test.shape)

# 7) QS Features

qs_uc_test = featureExtraction.QS(UC_documents_test, code_documents_test).reshape(1,-1) 
qs_code_test = featureExtraction.QS(code_documents_test, UC_documents_test).reshape(1,-1) 

print('qs_uc_test_shape', qs_uc_test.shape)
print('qs_code_test_shape', qs_code_test.shape)

# 8) SCS Features
UC_SCS_test = featureExtraction.simplifiedClarityScore(UC_documents_test, UC_count_matrix_test, tf_code_dict_test).reshape(1,-1) 
CC_SCS_test = featureExtraction.simplifiedClarityScore(code_documents_test, code_count_matrix_test, tf_uc_dict_test).reshape(1,-1) 

print('UC_SCS_test_shape', UC_SCS_test.shape)
print('CC_SCS_test_shape', CC_SCS_test.shape)

# 9) Coherence Score Features
UC_CoherenceScore_test = featureExtraction.CoherenceScore(UC_documents_test, tfidf_matrix_code_test).reshape(1,-1) 
CC_CoherenceScore_test = featureExtraction.CoherenceScore(code_documents_test, tfidf_matrix_uc_test).reshape(1,-1) 

print('UC_CoherenceScore_test_shape', UC_CoherenceScore_test.shape)
print('CC_CoherenceScore_test_shape', CC_CoherenceScore_test.shape)

#7mins

In [ ]:
#-------------------Saving test Preretival------------------#
np.save('./pickles/avg_idf_uc_test.npy', avg_idf_uc_test)
np.save('./pickles/avg_idf_code_test.npy', avg_idf_code_test)

np.save('./pickles/max_idf_uc_test.npy', max_idf_uc_test)
np.save('./pickles/max_idf_code_test.npy', max_idf_code_test)

np.save('./pickles/dev_idf_uc_test.npy', dev_idf_uc_test)
np.save('./pickles/dev_idf_code_test.npy', dev_idf_code_test)

np.save('./pickles/avg_ictf_uc_test.npy', avg_ictf_uc_test)
np.save('./pickles/avg_ictf_code_test.npy', avg_ictf_code_test)

np.save('./pickles/max_ictf_uc_test.npy', max_ictf_uc_test)
np.save('./pickles/max_ictf_code_test.npy', max_ictf_code_test)

np.save('./pickles/dev_ictf_uc_test.npy', dev_ictf_uc_test)
np.save('./pickles/dev_ictf_code_test.npy', dev_ictf_code_test)

np.save('./pickles/avg_entropy_uc_test.npy', avg_entropy_uc_test)
np.save('./pickles/avg_entropy_code_test.npy', avg_entropy_code_test)

np.save('./pickles/max_entropy_uc_test.npy', max_entropy_uc_test)
np.save('./pickles/max_entropy_code_test.npy', max_entropy_code_test)

np.save('./pickles/med_entropy_uc_test.npy', med_entropy_uc_test)
np.save('./pickles/med_entropy_code_test.npy', med_entropy_code_test)

np.save('./pickles/dev_entropy_uc_test.npy', dev_entropy_uc_test)
np.save('./pickles/dev_entropy_code_test.npy', dev_entropy_code_test)

np.save('./pickles/avg_variance_uc_test.npy', avg_variance_uc_test)
np.save('./pickles/avg_variance_code_test.npy', avg_variance_code_test)

np.save('./pickles/max_variance_uc_test.npy', max_variance_uc_test)
np.save('./pickles/max_variance_code_test.npy', max_variance_code_test)

np.save('./pickles/sum_variance_uc_test.npy', sum_variance_uc_test)
np.save('./pickles/sum_variance_code_test.npy', sum_variance_code_test)

np.save('./pickles/avg_scq_uc_test.npy', avg_scq_uc_test)
np.save('./pickles/avg_scq_code_test.npy', avg_scq_code_test)

np.save('./pickles/max_scq_uc_test.npy', max_scq_uc_test)
np.save('./pickles/max_scq_code_test.npy', max_scq_code_test)

np.save('./pickles/sum_sqc_uc_test.npy', sum_sqc_uc_test)
np.save('./pickles/sum_sqc_code_test.npy', sum_sqc_code_test)

np.save('./pickles/avg_pmi_uc_test.npy', avg_pmi_uc_test)
np.save('./pickles/avg_pmi_code_test.npy', avg_pmi_code_test)

np.save('./pickles/max_pmi_uc_test.npy', max_pmi_uc_test)
np.save('./pickles/max_pmi_code_test.npy', max_pmi_code_test)

np.save('./pickles/qs_uc_test.npy', qs_uc_test)
np.save('./pickles/qs_code_test.npy', qs_code_test)

np.save('./pickles/UC_SCS_test.npy', UC_SCS_test)
np.save('./pickles/CC_SCS_test.npy', CC_SCS_test)

np.save('./pickles/UC_CoherenceScore_test.npy', UC_CoherenceScore_test)
np.save('./pickles/CC_CoherenceScore_test.npy', CC_CoherenceScore_test)

In [16]:
# ------------------------pre-retrieval (21 metrics) Train--------------------------#

avg_idf_uc_train = np.load('./pickles/avg_idf_uc_train.npy')

avg_idf_uc_train = normalizer.fit_transform(avg_idf_uc_train.reshape(-1,1))

avg_idf_code_train = np.load('./pickles/avg_idf_code_train.npy')
avg_idf_code_train = normalizer.fit_transform(avg_idf_code_train.reshape(-1,1))

max_idf_uc_train = np.load('./pickles/max_idf_uc_train.npy')

max_idf_uc_train = normalizer.fit_transform(max_idf_uc_train.reshape(-1,1))

max_idf_code_train = np.load('./pickles/max_idf_code_train.npy')
max_idf_code_train = normalizer.fit_transform(max_idf_code_train.reshape(-1,1))

dev_idf_uc_train = np.load('./pickles/dev_idf_uc_train.npy')
dev_idf_uc_train = normalizer.fit_transform(dev_idf_uc_train.reshape(-1,1))

dev_idf_code_train = np.load('./pickles/dev_idf_code_train.npy')
dev_idf_code_train = normalizer.fit_transform(dev_idf_code_train.reshape(-1,1))

avg_ictf_uc_train = np.load('./pickles/avg_ictf_uc_train.npy')
avg_ictf_uc_train = normalizer.fit_transform(avg_ictf_uc_train.reshape(-1,1))

avg_ictf_code_train = np.load('./pickles/avg_ictf_code_train.npy')
avg_ictf_code_train = normalizer.fit_transform(avg_ictf_code_train.reshape(-1,1))

max_ictf_uc_train = np.load('./pickles/max_ictf_uc_train.npy')
max_ictf_uc_train = normalizer.fit_transform(max_ictf_uc_train.reshape(-1,1))

max_ictf_code_train = np.load('./pickles/max_ictf_code_train.npy')
max_ictf_code_train = normalizer.fit_transform(max_ictf_code_train.reshape(-1,1))

dev_ictf_uc_train = np.load('./pickles/dev_ictf_uc_train.npy')
dev_ictf_uc_train = normalizer.fit_transform(dev_ictf_uc_train.reshape(-1,1))

dev_ictf_code_train = np.load('./pickles/dev_ictf_code_train.npy')
dev_ictf_code_train = normalizer.fit_transform(dev_ictf_code_train.reshape(-1,1))

avg_entropy_uc_train = np.load('./pickles/avg_entropy_uc_train.npy')
avg_entropy_uc_train = normalizer.fit_transform(avg_entropy_uc_train.reshape(-1,1))

avg_entropy_code_train = np.load('./pickles/avg_entropy_code_train.npy')
avg_entropy_code_train = normalizer.fit_transform(avg_entropy_code_train.reshape(-1,1))

max_entropy_uc_train = np.load('./pickles/max_entropy_uc_train.npy')
max_entropy_uc_train = normalizer.fit_transform(max_entropy_uc_train.reshape(-1,1))

max_entropy_code_train = np.load('./pickles/max_entropy_code_train.npy')
max_entropy_code_train = normalizer.fit_transform(max_entropy_code_train.reshape(-1,1))

med_entropy_uc_train = np.load('./pickles/med_entropy_uc_train.npy')
med_entropy_uc_train = normalizer.fit_transform(med_entropy_uc_train.reshape(-1,1))
med_entropy_code_train = np.load('./pickles/med_entropy_code_train.npy')
med_entropy_code_train = normalizer.fit_transform(med_entropy_code_train.reshape(-1,1))

dev_entropy_uc_train = np.load('./pickles/dev_entropy_uc_train.npy')
dev_entropy_uc_train = normalizer.fit_transform(dev_entropy_uc_train.reshape(-1,1))

dev_entropy_code_train = np.load('./pickles/dev_entropy_code_train.npy')
dev_entropy_code_train = normalizer.fit_transform(dev_entropy_code_train.reshape(-1,1))

avg_variance_uc_train = np.load('./pickles/avg_variance_uc_train.npy')
avg_variance_uc_train = normalizer.fit_transform(avg_variance_uc_train.reshape(-1,1))

avg_variance_code_train = np.load('./pickles/avg_variance_code_train.npy')
avg_variance_code_train = normalizer.fit_transform(avg_variance_code_train.reshape(-1,1))

max_variance_uc_train = np.load('./pickles/max_variance_uc_train.npy')
max_variance_uc_train = normalizer.fit_transform(max_variance_uc_train.reshape(-1,1))

max_variance_code_train = np.load('./pickles/max_variance_code_train.npy')
max_variance_code_train = normalizer.fit_transform(max_variance_code_train.reshape(-1,1))
sum_variance_uc_train = np.load('./pickles/sum_variance_uc_train.npy')
sum_variance_uc_train = normalizer.fit_transform(sum_variance_uc_train.reshape(-1,1))

sum_variance_code_train = np.load('./pickles/sum_variance_code_train.npy')
sum_variance_code_train = normalizer.fit_transform(sum_variance_code_train.reshape(-1,1))

avg_scq_uc_train = np.load('./pickles/avg_scq_uc_train.npy')
avg_scq_uc_train = normalizer.fit_transform(avg_scq_uc_train.reshape(-1,1))

avg_scq_code_train = np.load('./pickles/avg_scq_code_train.npy')
avg_scq_code_train = normalizer.fit_transform(avg_scq_code_train.reshape(-1,1))

max_scq_uc_train = np.load('./pickles/max_scq_uc_train.npy')
max_scq_uc_train = normalizer.fit_transform(max_scq_uc_train.reshape(-1,1).reshape(-1,1))

max_scq_code_train = np.load('./pickles/max_scq_code_train.npy')
max_scq_code_train = normalizer.fit_transform(max_scq_code_train.reshape(-1,1))

sum_sqc_uc_train = np.load('./pickles/sum_sqc_uc_train.npy')
sum_sqc_uc_train = normalizer.fit_transform(sum_sqc_uc_train.reshape(-1,1))

sum_sqc_code_train = np.load('./pickles/sum_sqc_code_train.npy')
sum_sqc_code_train = normalizer.fit_transform(sum_sqc_code_train.reshape(-1,1))

avg_pmi_uc_train = np.load('./pickles/avg_pmi_uc_train.npy')
avg_pmi_uc_train = normalizer.fit_transform(avg_pmi_uc_train.reshape(-1,1))

avg_pmi_code_train = np.load('./pickles/avg_pmi_code_train.npy')
avg_pmi_code_train = normalizer.fit_transform(avg_pmi_code_train.reshape(-1,1))

max_pmi_uc_train = np.load('./pickles/max_pmi_uc_train.npy')
max_pmi_uc_train = normalizer.fit_transform(max_pmi_uc_train.reshape(-1,1))


max_pmi_code_train = np.load('./pickles/max_pmi_code_train.npy')
max_pmi_code_train = normalizer.fit_transform(max_pmi_code_train.reshape(-1,1))

qs_uc_train = np.load('./pickles/qs_uc_train.npy')
qs_uc_train = normalizer.fit_transform(qs_uc_train.reshape(-1,1))

qs_code_train = np.load('./pickles/qs_code_train.npy')
qs_code_train = normalizer.fit_transform(qs_code_train.reshape(-1,1))

UC_SCS_train = np.load('./pickles/UC_SCS_train.npy')
UC_SCS_train = normalizer.fit_transform(UC_SCS_train.reshape(-1,1))

CC_SCS_train = np.load('./pickles/CC_SCS_train.npy')
CC_SCS_train = normalizer.fit_transform(CC_SCS_train.reshape(-1,1))

UC_CoherenceScore_train = np.load('./pickles/UC_CoherenceScore_train.npy')
UC_CoherenceScore_train = normalizer.fit_transform(UC_CoherenceScore_train.reshape(-1,1))


CC_CoherenceScore_train = np.load('./pickles/CC_CoherenceScore_train.npy')
CC_CoherenceScore_train = normalizer.fit_transform(CC_CoherenceScore_train.reshape(-1,1))
print("max_ictf_code_train:", max_ictf_code_train) #gives ones only


max_ictf_code_train: [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [17]:
# ------------------------pre-retrieval (21 metrics) Test--------------------------#
avg_idf_uc_test = np.load('./pickles/avg_idf_uc_test.npy')
avg_idf_uc_test = normalizer.fit_transform(avg_idf_uc_test.reshape(-1,1))

avg_idf_code_test = np.load('./pickles/avg_idf_code_test.npy')
avg_idf_code_test = normalizer.fit_transform(avg_idf_code_test.reshape(-1,1))

max_idf_uc_test = np.load('./pickles/max_idf_uc_test.npy')
max_idf_uc_test = normalizer.fit_transform(max_idf_uc_test.reshape(-1,1))

max_idf_code_test = np.load('./pickles/max_idf_code_test.npy')
max_idf_code_test = normalizer.fit_transform(max_idf_code_test.reshape(-1,1))

dev_idf_uc_test = np.load('./pickles/dev_idf_uc_test.npy')
dev_idf_uc_test = normalizer.fit_transform(dev_idf_uc_test.reshape(-1,1))

dev_idf_code_test = np.load('./pickles/dev_idf_code_test.npy')
dev_idf_code_test = normalizer.fit_transform(dev_idf_code_test.reshape(-1,1))

avg_ictf_uc_test = np.load('./pickles/avg_ictf_uc_test.npy')
avg_ictf_uc_test = normalizer.fit_transform(avg_ictf_uc_test.reshape(-1,1))

avg_ictf_code_test = np.load('./pickles/avg_ictf_code_test.npy')
avg_ictf_code_test = normalizer.fit_transform(avg_ictf_code_test.reshape(-1,1))

max_ictf_uc_test = np.load('./pickles/max_ictf_uc_test.npy')
max_ictf_uc_test = normalizer.fit_transform(max_ictf_uc_test.reshape(-1,1))

max_ictf_code_test = np.load('./pickles/max_ictf_code_test.npy')
max_ictf_code_test = normalizer.fit_transform(max_ictf_code_test.reshape(-1,1))

dev_ictf_uc_test = np.load('./pickles/dev_ictf_uc_test.npy')
dev_ictf_uc_test = normalizer.fit_transform(dev_ictf_uc_test.reshape(-1,1))

dev_ictf_code_test = np.load('./pickles/dev_ictf_code_test.npy')
dev_ictf_code_test = normalizer.fit_transform(dev_ictf_code_test.reshape(-1,1))

avg_entropy_uc_test = np.load('./pickles/avg_entropy_uc_test.npy')
avg_entropy_uc_test = normalizer.fit_transform(avg_entropy_uc_test.reshape(-1,1))

avg_entropy_code_test = np.load('./pickles/avg_entropy_code_test.npy')
avg_entropy_code_test = normalizer.fit_transform(avg_entropy_code_test.reshape(-1,1))

max_entropy_uc_test = np.load('./pickles/max_entropy_uc_test.npy')
max_entropy_uc_test = normalizer.fit_transform(max_entropy_uc_test.reshape(-1,1))

max_entropy_code_test = np.load('./pickles/max_entropy_code_test.npy')
max_entropy_code_test = normalizer.fit_transform(max_entropy_code_test.reshape(-1,1))

med_entropy_uc_test = np.load('./pickles/med_entropy_uc_test.npy')
med_entropy_uc_test = normalizer.fit_transform(med_entropy_uc_test.reshape(-1,1))

med_entropy_code_test = np.load('./pickles/med_entropy_code_test.npy')
med_entropy_code_test = normalizer.fit_transform(med_entropy_code_test.reshape(-1,1))

dev_entropy_uc_test = np.load('./pickles/dev_entropy_uc_test.npy')
dev_entropy_uc_test = normalizer.fit_transform(dev_entropy_uc_test.reshape(-1,1))

dev_entropy_code_test = np.load('./pickles/dev_entropy_code_test.npy')
dev_entropy_code_test = normalizer.fit_transform(dev_entropy_code_test.reshape(-1,1))

avg_variance_uc_test = np.load('./pickles/avg_variance_uc_test.npy')
avg_variance_uc_test = normalizer.fit_transform(avg_variance_uc_test.reshape(-1,1))

avg_variance_code_test = np.load('./pickles/avg_variance_code_test.npy')
avg_variance_code_test = normalizer.fit_transform(avg_variance_code_test.reshape(-1,1))

max_variance_uc_test = np.load('./pickles/max_variance_uc_test.npy')
max_variance_uc_test = normalizer.fit_transform(max_variance_uc_test.reshape(-1,1))

max_variance_code_test = np.load('./pickles/max_variance_code_test.npy')
max_variance_code_test = normalizer.fit_transform(max_variance_code_test.reshape(-1,1))

sum_variance_uc_test = np.load('./pickles/sum_variance_uc_test.npy')
sum_variance_uc_test = normalizer.fit_transform(sum_variance_uc_test.reshape(-1,1))

sum_variance_code_test = np.load('./pickles/sum_variance_code_test.npy')
sum_variance_code_test = normalizer.fit_transform(sum_variance_code_test.reshape(-1,1))

avg_scq_uc_test = np.load('./pickles/avg_scq_uc_test.npy')
avg_scq_uc_test = normalizer.fit_transform(avg_scq_uc_test.reshape(-1,1))

avg_scq_code_test = np.load('./pickles/avg_scq_code_test.npy')
avg_scq_code_test = normalizer.fit_transform(avg_scq_code_test.reshape(-1,1))

max_scq_uc_test = np.load('./pickles/max_scq_uc_test.npy')
max_scq_uc_test = normalizer.fit_transform(max_scq_uc_test.reshape(-1,1))

max_scq_code_test = np.load('./pickles/max_scq_code_test.npy')
max_scq_code_test = normalizer.fit_transform(max_scq_code_test.reshape(-1,1))

sum_sqc_uc_test = np.load('./pickles/sum_sqc_uc_test.npy')
sum_sqc_uc_test = normalizer.fit_transform(sum_sqc_uc_test.reshape(-1,1))

sum_sqc_code_test = np.load('./pickles/sum_sqc_code_test.npy')
sum_sqc_code_test = normalizer.fit_transform(sum_sqc_code_test.reshape(-1,1))

avg_pmi_uc_test = np.load('./pickles/avg_pmi_uc_test.npy')
avg_pmi_uc_test = normalizer.fit_transform(avg_pmi_uc_test.reshape(-1,1))

avg_pmi_code_test = np.load('./pickles/avg_pmi_code_test.npy')
avg_pmi_code_test = normalizer.fit_transform(avg_pmi_code_test.reshape(-1,1))

max_pmi_uc_test = np.load('./pickles/max_pmi_uc_test.npy')
max_pmi_uc_test = normalizer.fit_transform(max_pmi_uc_test.reshape(-1,1))

max_pmi_code_test = np.load('./pickles/max_pmi_code_test.npy')
max_pmi_code_test = normalizer.fit_transform(max_pmi_code_test.reshape(-1,1))

qs_uc_test = np.load('./pickles/qs_uc_test.npy')
qs_uc_test = normalizer.fit_transform(qs_uc_test.reshape(-1,1))

qs_code_test = np.load('./pickles/qs_code_test.npy')
qs_code_test = normalizer.fit_transform(qs_code_test.reshape(-1,1))

UC_SCS_test = np.load('./pickles/UC_SCS_test.npy')
UC_SCS_test = normalizer.fit_transform(UC_SCS_test.reshape(-1,1))

CC_SCS_test = np.load('./pickles/CC_SCS_test.npy')
CC_SCS_test = normalizer.fit_transform(CC_SCS_test.reshape(-1,1))

UC_CoherenceScore_test = np.load('./pickles/UC_CoherenceScore_test.npy')
UC_CoherenceScore_test = normalizer.fit_transform(UC_CoherenceScore_test.reshape(-1,1))

CC_CoherenceScore_test = np.load('./pickles/CC_CoherenceScore_test.npy')
CC_CoherenceScore_test = normalizer.fit_transform(CC_CoherenceScore_test.reshape(-1,1))


### Post Retrieval features

In [ ]:
#BASSANT COMMENTED THIS
# #------------------------post-retrieval (7 metrics) Train --------------------------#
# # 1.1) Subquery overlap using jensenShannon
# code_queries_score_JensenShannon_train = featureExtraction.SubqueryOverlap(code_documents_train, featureExtraction.JensenShannon,"JS", UC_count_matrix_train, code_count_matrix_train)
# UC_queries_score_JensenShannon_train = featureExtraction.SubqueryOverlap(UC_documents_train, featureExtraction.JensenShannon,"JS",code_count_matrix_train, UC_count_matrix_train)

# print('code_queries_score_JensenShannon_train', code_queries_score_JensenShannon_train.shape)
# print('UC_queries_score_JensenShannon_train', UC_queries_score_JensenShannon_train.shape)

# # 1.2) Subquery overlap using VSM
# code_queries_score_VSM_train = featureExtraction.SubqueryOverlap(code_documents_train, featureExtraction.VectorSpaceModel,"VSM", UC_count_matrix_train, code_count_matrix_train)
# UC_queries_score_VSM_train = featureExtraction.SubqueryOverlap(UC_documents_train, featureExtraction.VectorSpaceModel,"VSM", code_count_matrix_train, UC_count_matrix_train)

# print('code_queries_score_VSM_train', code_queries_score_VSM_train.shape)
# print('UC_queries_score_VSM_train', UC_queries_score_VSM_train.shape)

# #1.3) Subquery overlap using BM25
# code_queries_score_BM25_train = featureExtraction.SubqueryOverlap(code_documents_train, featureExtraction.BM25,"BM", UC_documents_train,idf_uc_dict_train,UC_count_matrix_train)
# UC_queries_score_BM25_train = featureExtraction.SubqueryOverlap(UC_documents_train, featureExtraction.BM25,"BM", code_documents_train,idf_code_dict_train, code_count_matrix_train)

# print('code_queries_score_BM25_train', code_queries_score_BM25_train.shape)
# print('UC_queries_score_BM25_train', UC_queries_score_BM25_train.shape)

# #1.4) Subquery overlap using JM smoothing
# code_queries_score_JM_train = featureExtraction.SubqueryOverlap(code_documents_train, featureExtraction.SmoothingMethods,"SM", UC_documents_train,UC_count_matrix_train,tf_uc_dict_train,True)
# UC_queries_score_JM_train = featureExtraction.SubqueryOverlap(UC_documents_train, featureExtraction.SmoothingMethods,"SM", code_documents_train,code_count_matrix_train,tf_code_dict_train,True)

# print('code_queries_score_JM_train', code_queries_score_JM_train.shape)
# print('UC_queries_score_JM_train', UC_queries_score_JM_train.shape)

# # 1.5) Subquery overlap using DP smoothing
# code_queries_score_DP_train = featureExtraction.SubqueryOverlap(code_documents_train, featureExtraction.SmoothingMethods,"SM", UC_documents_train, UC_count_matrix_train,tf_uc_dict_train,False)
# UC_queries_score_DP_train = featureExtraction.SubqueryOverlap(UC_documents_train, featureExtraction.SmoothingMethods,"SM", code_documents_train, code_count_matrix_train,tf_code_dict_train,False)

# print('code_queries_score_DP_train', code_queries_score_DP_train.shape)
# print('UC_queries_score_DP_train', UC_queries_score_DP_train.shape)

# #------------------------post-retrieval (7 metrics) Test --------------------------#
# # 1.1) Subquery overlap using jensenShannon
# code_queries_score_JensenShannon_test = featureExtraction.SubqueryOverlap(code_documents_test, featureExtraction.JensenShannon,"JS", UC_count_matrix_test, code_count_matrix_test)
# UC_queries_score_JensenShannon_test = featureExtraction.SubqueryOverlap(UC_documents_test, featureExtraction.JensenShannon,"JS",code_count_matrix_test, UC_count_matrix_test)

# print('code_queries_score_JensenShannon_test', code_queries_score_JensenShannon_test.shape)
# print('UC_queries_score_JensenShannon_test', UC_queries_score_JensenShannon_test.shape)

# # 1.2) Subquery overlap using VSM
# code_queries_score_VSM_test = featureExtraction.SubqueryOverlap(code_documents_test, featureExtraction.VectorSpaceModel,"VSM", UC_count_matrix_test, code_count_matrix_test)
# UC_queries_score_VSM_test = featureExtraction.SubqueryOverlap(UC_documents_test, featureExtraction.VectorSpaceModel,"VSM", code_count_matrix_test, UC_count_matrix_test)

# print('code_queries_score_VSM_test', code_queries_score_VSM_test.shape)
# print('UC_queries_score_VSM_test', UC_queries_score_VSM_test.shape)

# #1.3) Subquery overlap using BM25
# code_queries_score_BM25_test = featureExtraction.SubqueryOverlap(code_documents_test, featureExtraction.BM25,"BM", UC_documents_test,idf_uc_dict_test,UC_count_matrix_test)
# UC_queries_score_BM25_test = featureExtraction.SubqueryOverlap(UC_documents_test, featureExtraction.BM25,"BM", code_documents_test,idf_code_dict_test, code_count_matrix_test)

# print('code_queries_score_BM25_test', code_queries_score_BM25_test.shape)
# print('UC_queries_score_BM25_test', UC_queries_score_BM25_test.shape)

# #1.4) Subquery overlap using JM smoothing
# code_queries_score_JM_test = featureExtraction.SubqueryOverlap(code_documents_test, featureExtraction.SmoothingMethods,"SM", UC_documents_test,UC_count_matrix_test,tf_uc_dict_test,True)
# UC_queries_score_JM_test = featureExtraction.SubqueryOverlap(UC_documents_test, featureExtraction.SmoothingMethods,"SM", code_documents_test,code_count_matrix_test,tf_code_dict_test,True)

# print('code_queries_score_JM_test', code_queries_score_JM_test.shape)
# print('UC_queries_score_JM_test', UC_queries_score_JM_test.shape)

# # 1.5) Subquery overlap using DP smoothing
# code_queries_score_DP_test = featureExtraction.SubqueryOverlap(code_documents_test, featureExtraction.SmoothingMethods,"SM", UC_documents_test, UC_count_matrix_test,tf_uc_dict_test,False)
# UC_queries_score_DP_test = featureExtraction.SubqueryOverlap(UC_documents_test, featureExtraction.SmoothingMethods,"SM", code_documents_test, code_count_matrix_test,tf_code_dict_test,False)

# print('code_queries_score_DP_test', code_queries_score_DP_test.shape)
# print('UC_queries_score_DP_test', UC_queries_score_DP_test.shape)

In [7]:
#--------------------------------Robustness Score Train-------------------------------------------------------#

# 2.1) Robustness Score using jensenShannon
UC_RS_JS_train, UC_FRC_JS_train = featureExtraction.RobustnessScore(UC_documents_train,code_documents_train,"JS")
code_RS_JS_train, code_FRC_JS_train = featureExtraction.RobustnessScore(code_documents_train,UC_documents_train,"JS")

print('UC_RS_JS_train', UC_RS_JS_train.shape)
print('code_RS_JS_train', code_RS_JS_train.shape)

print('UC_FRC_JS_train', UC_FRC_JS_train.shape)
print('code_FRC_JS_train', code_FRC_JS_train.shape)

np.save('./pickles/UC_RS_JS_train.npy', UC_RS_JS_train)
np.save('./pickles/code_RS_JS_train.npy', code_RS_JS_train)

np.save('./pickles/UC_FRC_JS_train.npy', UC_FRC_JS_train)
np.save('./pickles/code_FRC_JS_train.npy', code_FRC_JS_train)

# 2.2) Robustness Score using VSM
UC_RS_VSM_train, UC_FRC_VSM_train = featureExtraction.RobustnessScore(UC_documents_train,code_documents_train,"VSM")
code_RS_VSM_train, code_FRC_VSM_train = featureExtraction.RobustnessScore(code_documents_train,UC_documents_train,"VSM")

print('UC_RS_VSM_train', UC_RS_VSM_train.shape)
print('code_RS_VSM_train', code_RS_VSM_train.shape)

print('UC_FRC_VSM_train', UC_FRC_VSM_train.shape)
print('code_FRC_VSM_train', code_FRC_VSM_train.shape)

np.save('./pickles/UC_RS_VSM_train.npy', UC_RS_VSM_train)
np.save('./pickles/code_RS_VSM_train.npy', code_RS_VSM_train)

np.save('./pickles/UC_FRC_VSM_train.npy', UC_FRC_VSM_train)
np.save('./pickles/code_FRC_VSM_train.npy', code_FRC_VSM_train)

# 2.3) Robustness Score using BM25
UC_RS_BM25_train, UC_FRC_BM25_train = featureExtraction.RobustnessScore(UC_documents_train,code_documents_train,"BM")
code_RS_BM25_train, code_FRC_BM25_train = featureExtraction.RobustnessScore(code_documents_train,UC_documents_train,"BM")

print('UC_RS_BM25_train', UC_RS_BM25_train.shape)
print('code_RS_BM25_train', code_RS_BM25_train.shape)

print('UC_FRC_BM25_train', UC_FRC_BM25_train.shape)
print('code_FRC_BM25_train', code_FRC_BM25_train.shape)

np.save('./pickles/UC_RS_BM25_train.npy', UC_RS_BM25_train)
np.save('./pickles/code_RS_BM25_train.npy', code_RS_BM25_train)

np.save('./pickles/UC_FRC_BM25_train.npy', UC_FRC_BM25_train)
np.save('./pickles/code_FRC_BM25_train.npy', code_FRC_BM25_train)

# 2.4) Robustness Score using JM smoothing
UC_RS_JM_train, UC_FRC_JM_train = featureExtraction.RobustnessScore(UC_documents_train,code_documents_train,"JM")
code_RS_JM_train, code_FRC_JM_train = featureExtraction.RobustnessScore(code_documents_train,UC_documents_train,"JM")

print('UC_RS_JM_train', UC_RS_JM_train.shape)
print('code_RS_JM_train', code_RS_JM_train.shape)

print('UC_FRC_JM_train', UC_FRC_JM_train.shape)
print('code_FRC_JM_train', code_FRC_JM_train.shape)

np.save('./pickles/UC_RS_JM_train.npy', UC_RS_JM_train)
np.save('./pickles/code_RS_JM_train.npy', code_RS_JM_train)

np.save('./pickles/UC_FRC_JM_train.npy', UC_FRC_JM_train)
np.save('./pickles/code_FRC_JM_train.npy', code_FRC_JM_train)

# 2.5) Robustness Score using DP smoothing
UC_RS_DP_train, UC_FRC_DP_train = featureExtraction.RobustnessScore(UC_documents_train,code_documents_train,"DP")
code_RS_DP_train, code_FRC_DP_train = featureExtraction.RobustnessScore(code_documents_train,UC_documents_train,"DP")

print('UC_RS_DP_train', UC_RS_DP_train.shape)
print('code_RS_DP_train', code_RS_DP_train.shape)

print('UC_FRC_DP_train', UC_FRC_DP_train.shape)
print('code_FRC_DP_train', code_FRC_DP_train.shape)

np.save('./pickles/UC_RS_DP_train.npy', UC_RS_DP_train)
np.save('./pickles/code_RS_DP_train.npy', code_RS_DP_train)

np.save('./pickles/UC_FRC_DP_train.npy', UC_FRC_DP_train)
np.save('./pickles/code_FRC_DP_train.npy', code_FRC_DP_train)

UC_RS_JS_train (2398,)
code_RS_JS_train (1031,)
UC_FRC_JS_train (2398,)
code_FRC_JS_train (1031,)
UC_RS_VSM_train (2398,)
code_RS_VSM_train (1031,)
UC_FRC_VSM_train (2398,)
code_FRC_VSM_train (1031,)
UC_RS_BM25_train (2398,)
code_RS_BM25_train (1031,)
UC_FRC_BM25_train (2398,)
code_FRC_BM25_train (1031,)
UC_RS_JM_train (2398,)
code_RS_JM_train (1031,)
UC_FRC_JM_train (2398,)
code_FRC_JM_train (1031,)
UC_RS_DP_train (2398,)
code_RS_DP_train (1031,)
UC_FRC_DP_train (2398,)
code_FRC_DP_train (1031,)


In [18]:
UC_RS_JS_train = np.load('./pickles/UC_RS_JS_train.npy')
UC_RS_JS_train = normalizer.fit_transform(UC_RS_JS_train.reshape(-1, 1))

code_RS_JS_train = np.load('./pickles/code_RS_JS_train.npy')
code_RS_JS_train = normalizer.fit_transform(code_RS_JS_train.reshape(-1, 1))

UC_FRC_JS_train = np.load('./pickles/UC_FRC_JS_train.npy')
UC_FRC_JS_train = normalizer.fit_transform(UC_FRC_JS_train.reshape(-1, 1))

code_FRC_JS_train = np.load('./pickles/code_FRC_JS_train.npy')
code_FRC_JS_train = normalizer.fit_transform(code_FRC_JS_train.reshape(-1, 1))

UC_RS_VSM_train = np.load('./pickles/UC_RS_VSM_train.npy')
UC_RS_VSM_train = normalizer.fit_transform(UC_RS_VSM_train.reshape(-1, 1))

code_RS_VSM_train = np.load('./pickles/code_RS_VSM_train.npy')
code_RS_VSM_train = normalizer.fit_transform(code_RS_VSM_train.reshape(-1, 1))

UC_FRC_VSM_train = np.load('./pickles/UC_FRC_VSM_train.npy')
UC_FRC_VSM_train = normalizer.fit_transform(UC_FRC_VSM_train.reshape(-1, 1))

code_FRC_VSM_train = np.load('./pickles/code_FRC_VSM_train.npy')
code_FRC_VSM_train = normalizer.fit_transform(code_FRC_VSM_train.reshape(-1, 1))

UC_RS_BM25_train = np.load('./pickles/UC_RS_BM25_train.npy')
UC_RS_BM25_train = normalizer.fit_transform(UC_RS_BM25_train.reshape(-1, 1))

code_RS_BM25_train = np.load('./pickles/code_RS_BM25_train.npy')
code_RS_BM25_train = normalizer.fit_transform(code_RS_BM25_train.reshape(-1, 1))

UC_FRC_BM25_train = np.load('./pickles/UC_FRC_BM25_train.npy')
UC_FRC_BM25_train = normalizer.fit_transform(UC_FRC_BM25_train.reshape(-1, 1))

code_FRC_BM25_train = np.load('./pickles/code_FRC_BM25_train.npy')
code_FRC_BM25_train = normalizer.fit_transform(code_FRC_BM25_train.reshape(-1, 1))

UC_RS_JM_train = np.load('./pickles/UC_RS_JM_train.npy')
UC_RS_JM_train = normalizer.fit_transform(UC_RS_JM_train.reshape(-1, 1))

code_RS_JM_train = np.load('./pickles/code_RS_JM_train.npy')
code_RS_JM_train = normalizer.fit_transform(code_RS_JM_train.reshape(-1, 1))

UC_FRC_JM_train = np.load('./pickles/UC_FRC_JM_train.npy')
UC_FRC_JM_train = normalizer.fit_transform(UC_FRC_JM_train.reshape(-1, 1))

code_FRC_JM_train = np.load('./pickles/code_FRC_JM_train.npy')
code_FRC_JM_train = normalizer.fit_transform(code_FRC_JM_train.reshape(-1, 1))

UC_RS_DP_train = np.load('./pickles/UC_RS_DP_train.npy')
UC_RS_DP_train = normalizer.fit_transform(UC_RS_DP_train.reshape(-1, 1))

code_RS_DP_train = np.load('./pickles/code_RS_DP_train.npy')
code_RS_DP_train = normalizer.fit_transform(code_RS_DP_train.reshape(-1, 1))

UC_FRC_DP_train = np.load('./pickles/UC_FRC_DP_train.npy')
UC_FRC_DP_train = normalizer.fit_transform(UC_FRC_DP_train.reshape(-1, 1))

code_FRC_DP_train = np.load('./pickles/code_FRC_DP_train.npy')
code_FRC_DP_train = normalizer.fit_transform(code_FRC_DP_train.reshape(-1, 1))

print(UC_FRC_DP_train,UC_FRC_JM_train,UC_FRC_BM25_train,UC_FRC_JS_train,UC_FRC_VSM_train) # ALERT : el hgat dy bytgeb zeros w ones


[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]] [[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]] [[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]] [[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]] [[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


In [8]:
#--------------------------------------------------Robustness Score test------------------------------------------------------------#

# 2.1) Robustness Score using jensenShannon
UC_RS_JS_test, UC_FRC_JS_test = featureExtraction.RobustnessScore(UC_documents_test,code_documents_test,"JS")
code_RS_JS_test, code_FRC_JS_test = featureExtraction.RobustnessScore(code_documents_test,UC_documents_test,"JS")

print('UC_RS_JS_test', UC_RS_JS_test.shape)
print('code_RS_JS_test', code_RS_JS_test.shape)

print('UC_FRC_JS_test', UC_FRC_JS_test.shape)
print('code_FRC_JS_test', code_FRC_JS_test.shape)

np.save('./pickles/UC_RS_JS_test.npy', UC_RS_JS_test)
np.save('./pickles/code_RS_JS_test.npy', code_RS_JS_test)

np.save('./pickles/UC_FRC_JS_test.npy', UC_FRC_JS_test)
np.save('./pickles/code_FRC_JS_test.npy', code_FRC_JS_test)

# 2.2) Robustness Score using VSM
UC_RS_VSM_test, UC_FRC_VSM_test = featureExtraction.RobustnessScore(UC_documents_test,code_documents_test,"VSM")
code_RS_VSM_test, code_FRC_VSM_test = featureExtraction.RobustnessScore(code_documents_test,UC_documents_test,"VSM")

print('UC_RS_VSM_test', UC_RS_VSM_test.shape)
print('code_RS_VSM_test', code_RS_VSM_test.shape)

print('UC_FRC_VSM_test', UC_FRC_VSM_test.shape)
print('code_FRC_VSM_test', code_FRC_VSM_test.shape)

np.save('./pickles/UC_RS_VSM_test.npy', UC_RS_VSM_test)
np.save('./pickles/code_RS_VSM_test.npy', code_RS_VSM_test)

np.save('./pickles/UC_FRC_VSM_test.npy', UC_FRC_VSM_test)
np.save('./pickles/code_FRC_VSM_test.npy', code_FRC_VSM_test)

# 2.3) Robustness Score using BM25
UC_RS_BM25_test, UC_FRC_BM25_test = featureExtraction.RobustnessScore(UC_documents_test,code_documents_test,"BM")
code_RS_BM25_test, code_FRC_BM25_test = featureExtraction.RobustnessScore(code_documents_test,UC_documents_test,"BM")

print('UC_RS_BM25_test', UC_RS_BM25_test.shape)
print('code_RS_BM25_test', code_RS_BM25_test.shape)

print('UC_FRC_BM25_test', UC_FRC_BM25_test.shape)
print('code_FRC_BM25_test', code_FRC_BM25_test.shape)

np.save('./pickles/UC_RS_BM25_test.npy', UC_RS_BM25_test)
np.save('./pickles/code_RS_BM25_test.npy', code_RS_BM25_test)

np.save('./pickles/UC_FRC_BM25_test.npy', UC_FRC_BM25_test)
np.save('./pickles/code_FRC_BM25_test.npy', code_FRC_BM25_test)

# 2.4) Robustness Score using JM smoothing
UC_RS_JM_test, UC_FRC_JM_test = featureExtraction.RobustnessScore(UC_documents_test,code_documents_test,"JM")
code_RS_JM_test, code_FRC_JM_test = featureExtraction.RobustnessScore(code_documents_test,UC_documents_test,"JM")

print('UC_RS_JM_test', UC_RS_JM_test.shape)
print('code_RS_JM_test', code_RS_JM_test.shape)

print('UC_FRC_JM_test', UC_FRC_JM_test.shape)
print('code_FRC_JM_test', code_FRC_JM_test.shape)

np.save('./pickles/UC_RS_JM_test.npy', UC_RS_JM_test)
np.save('./pickles/code_RS_JM_test.npy', code_RS_JM_test)

np.save('./pickles/UC_FRC_JM_test.npy', UC_FRC_JM_test)
np.save('./pickles/code_FRC_JM_test.npy', code_FRC_JM_test)

# 2.5) Robustness Score using DP smoothing
UC_RS_DP_test, UC_FRC_DP_test = featureExtraction.RobustnessScore(UC_documents_test,code_documents_test,"DP")
code_RS_DP_test, code_FRC_DP_test = featureExtraction.RobustnessScore(code_documents_test,UC_documents_test,"DP")

print('UC_RS_DP_test', UC_RS_DP_test.shape)
print('code_RS_DP_test', code_RS_DP_test.shape)

print('UC_FRC_DP_test', UC_FRC_DP_test.shape)
print('code_FRC_DP_test', code_FRC_DP_test.shape)

np.save('./pickles/UC_RS_DP_test.npy', UC_RS_DP_test)
np.save('./pickles/code_RS_DP_test.npy', code_RS_DP_test)

np.save('./pickles/UC_FRC_DP_test.npy', UC_FRC_DP_test)
np.save('./pickles/code_FRC_DP_test.npy', code_FRC_DP_test)

UC_RS_JS_test (409,)
code_RS_JS_test (174,)
UC_FRC_JS_test (409,)
code_FRC_JS_test (174,)
UC_RS_VSM_test (409,)
code_RS_VSM_test (174,)
UC_FRC_VSM_test (409,)
code_FRC_VSM_test (174,)
UC_RS_BM25_test (409,)
code_RS_BM25_test (174,)
UC_FRC_BM25_test (409,)
code_FRC_BM25_test (174,)
UC_RS_JM_test (409,)
code_RS_JM_test (174,)
UC_FRC_JM_test (409,)
code_FRC_JM_test (174,)
UC_RS_DP_test (409,)
code_RS_DP_test (174,)
UC_FRC_DP_test (409,)
code_FRC_DP_test (174,)


In [19]:
#ALERT :FRC BYTL3 1 AND ZEROS M3RFSH LW DY HAGA SAH
UC_RS_JS_test = np.load('./pickles/UC_RS_JS_test.npy')
UC_RS_JS_test = normalizer.fit_transform(UC_RS_JS_test.reshape(-1, 1))

code_RS_JS_test = np.load('./pickles/code_RS_JS_test.npy')
code_RS_JS_test = normalizer.fit_transform(code_RS_JS_test.reshape(-1, 1))

UC_FRC_JS_test = np.load('./pickles/UC_FRC_JS_test.npy')  #ALERT: bytl3 1 and zeros mfesh decimals
UC_FRC_JS_test = normalizer.fit_transform(UC_FRC_JS_test.reshape(-1, 1))

code_FRC_JS_test = np.load('./pickles/code_FRC_JS_test.npy')
code_FRC_JS_test = normalizer.fit_transform(code_FRC_JS_test.reshape(-1, 1))

UC_RS_VSM_test = np.load('./pickles/UC_RS_VSM_test.npy')
UC_RS_VSM_test = normalizer.fit_transform(UC_RS_VSM_test.reshape(-1, 1))

code_RS_VSM_test = np.load('./pickles/code_RS_VSM_test.npy')
code_RS_VSM_test = normalizer.fit_transform(code_RS_VSM_test.reshape(-1, 1))

UC_FRC_VSM_test = np.load('./pickles/UC_FRC_VSM_test.npy')
UC_FRC_VSM_test = normalizer.fit_transform(UC_FRC_VSM_test.reshape(-1, 1))

code_FRC_VSM_test = np.load('./pickles/code_FRC_VSM_test.npy')
code_FRC_VSM_test= normalizer.fit_transform(code_FRC_VSM_test.reshape(-1, 1))

UC_RS_BM25_test = np.load('./pickles/UC_RS_BM25_test.npy')
UC_RS_BM25_test = normalizer.fit_transform(UC_RS_BM25_test.reshape(-1, 1))

code_RS_BM25_test = np.load('./pickles/code_RS_BM25_test.npy')
code_RS_BM25_test = normalizer.fit_transform(code_RS_BM25_test.reshape(-1, 1))

UC_FRC_BM25_test = np.load('./pickles/UC_FRC_BM25_test.npy')
UC_FRC_BM25_test = normalizer.fit_transform(UC_FRC_BM25_test.reshape(-1, 1))

code_FRC_BM25_test = np.load('./pickles/code_FRC_BM25_test.npy')
code_FRC_BM25_test = normalizer.fit_transform(code_FRC_BM25_test.reshape(-1, 1))

UC_RS_JM_test = np.load('./pickles/UC_RS_JM_test.npy')
UC_RS_JM_test = normalizer.fit_transform(UC_RS_JM_test.reshape(-1, 1))

code_RS_JM_test = np.load('./pickles/code_RS_JM_test.npy')
code_RS_JM_test = normalizer.fit_transform(code_RS_JM_test.reshape(-1, 1))

UC_FRC_JM_test = np.load('./pickles/UC_FRC_JM_test.npy')
UC_FRC_JM_test = normalizer.fit_transform(UC_FRC_JM_test.reshape(-1, 1))

code_FRC_JM_test = np.load('./pickles/code_FRC_JM_test.npy')
code_FRC_JM_test = normalizer.fit_transform(code_FRC_JM_test.reshape(-1, 1))

UC_RS_DP_test = np.load('./pickles/UC_RS_DP_test.npy')
UC_RS_DP_test = normalizer.fit_transform(UC_RS_DP_test.reshape(-1, 1))

code_RS_DP_test = np.load('./pickles/code_RS_DP_test.npy')
code_RS_DP_test = normalizer.fit_transform(code_RS_DP_test.reshape(-1, 1))

UC_FRC_DP_test = np.load('./pickles/UC_FRC_DP_test.npy')
UC_FRC_DP_test = normalizer.fit_transform(UC_FRC_DP_test.reshape(-1, 1))

code_FRC_DP_test = np.load('./pickles/code_FRC_DP_test.npy')
code_FRC_DP_test = normalizer.fit_transform(code_FRC_DP_test.reshape(-1, 1))


In [17]:
#------------------------------------------------- ClusteringTendency Train---------------------------------------------
# 4.1) Clustering Tendency using JensenShannon
UC_CT_JensenShannon_train = featureExtraction.ClusteringTendency(JS_UC_train.T,tfidf_matrix_uc_train,tfidf_matrix_code_train)
code_CT_JensenShannon_train = featureExtraction.ClusteringTendency(JS_CC_train,tfidf_matrix_code_train,tfidf_matrix_uc_train)

print("UC_CT_JensenShannon_train",UC_CT_JensenShannon_train.shape)
print("code_CT_JensenShannon_train",code_CT_JensenShannon_train.shape)

np.save('./pickles/UC_CT_JensenShannon_train.npy',UC_CT_JensenShannon_train)
np.save('./pickles/code_CT_JensenShannon_train.npy',code_CT_JensenShannon_train)

# 4.2) Clustering Tendency using VSM
UC_CT_VSM_train = featureExtraction.ClusteringTendency(cosine_similarity_UC_train.T,tfidf_matrix_uc_train,tfidf_matrix_code_train)
code_CT_VSM_train = featureExtraction.ClusteringTendency(cosine_similarity_CC_train,tfidf_matrix_code_train,tfidf_matrix_uc_train)

print("UC_CT_VSM_train",UC_CT_VSM_train.shape)
print("code_CT_VSM_train",code_CT_VSM_train.shape)

np.save('./pickles/UC_CT_VSM_train.npy',UC_CT_VSM_train)
np.save('./pickles/code_CT_VSM_train.npy',code_CT_VSM_train)

# 4.3) Clustering Tendency using BM25
UC_CT_BM25_train = featureExtraction.ClusteringTendency(BM25_UC_train,tfidf_matrix_uc_train,tfidf_matrix_code_train)
code_CT_BM25_train = featureExtraction.ClusteringTendency(BM25_CC_train,tfidf_matrix_code_train,tfidf_matrix_uc_train)

print("UC_CT_BM25_train",UC_CT_BM25_train.shape)
print("code_CT_BM25_train",code_CT_BM25_train.shape)

np.save('./pickles/UC_CT_BM25_train.npy',UC_CT_BM25_train)
np.save('./pickles/code_CT_BM25_train.npy',code_CT_BM25_train)

# 4.4) Clustering Tendency using JM
UC_CT_JM_train = featureExtraction.ClusteringTendency(JM_UC_train,tfidf_matrix_uc_train,tfidf_matrix_code_train)
code_CT_JM_train = featureExtraction.ClusteringTendency(JM_CC_train,tfidf_matrix_code_train,tfidf_matrix_uc_train)

print("UC_CT_JM_train",UC_CT_JM_train.shape)
print("code_CT_JM_train",code_CT_JM_train.shape)


np.save('./pickles/UC_CT_JM_train.npy',UC_CT_JM_train)
np.save('./pickles/code_CT_JM_train.npy',code_CT_JM_train) 

# 4.5) Clustering Tendency using DP
UC_CT_DP_train = featureExtraction.ClusteringTendency(DP_UC_train,tfidf_matrix_uc_train,tfidf_matrix_code_train)
code_CT_DP_train = featureExtraction.ClusteringTendency(DP_CC_train,tfidf_matrix_code_train,tfidf_matrix_uc_train)

print("UC_CT_DP_train",UC_CT_DP_train.shape)
print("code_CT_DP_train",code_CT_DP_train.shape)

np.save('./pickles/UC_CT_DP_train.npy',UC_CT_DP_train)
np.save('./pickles/code_CT_DP_train.npy',code_CT_DP_train) 
#149.17min

UC_CT_JensenShannon_train (2398, 1)
code_CT_JensenShannon_train (1031, 1)
UC_CT_VSM_train (2398, 1)
code_CT_VSM_train (1031, 1)
UC_CT_BM25_train (2398, 1)
code_CT_BM25_train (1031, 1)
UC_CT_JM_train (2398, 1)
code_CT_JM_train (1031, 1)
UC_CT_DP_train (2398, 1)
code_CT_DP_train (1031, 1)


In [18]:
#----------------------------------ClusteringTendency test-------------------------------------------------
# 4.1) Clustering Tendency using JensenShannon
UC_CT_JensenShannon_test = featureExtraction.ClusteringTendency(JS_UC_test.T,tfidf_matrix_uc_test,tfidf_matrix_code_test)
code_CT_JensenShannon_test = featureExtraction.ClusteringTendency(JS_CC_test,tfidf_matrix_code_test,tfidf_matrix_uc_test)

print("UC_CT_JensenShannon_test",UC_CT_JensenShannon_test.shape)
print("code_CT_JensenShannon_test",code_CT_JensenShannon_test.shape)

np.save('./pickles/UC_CT_JensenShannon_test.npy',UC_CT_JensenShannon_test)
np.save('./pickles/code_CT_JensenShannon_test.npy',code_CT_JensenShannon_test) 

# 4.2) Clustering Tendency using VSM
UC_CT_VSM_test = featureExtraction.ClusteringTendency(cosine_similarity_UC_test.T,tfidf_matrix_uc_test,tfidf_matrix_code_test)
code_CT_VSM_test = featureExtraction.ClusteringTendency(cosine_similarity_CC_test,tfidf_matrix_code_test,tfidf_matrix_uc_test)

print("UC_CT_VSM_test",UC_CT_VSM_test.shape)
print("code_CT_VSM_test",code_CT_VSM_test.shape)

np.save('./pickles/UC_CT_VSM_test.npy',UC_CT_VSM_test)
np.save('./pickles/code_CT_VSM_test.npy',code_CT_VSM_test)

# 4.3) Clustering Tendency using BM25
UC_CT_BM25_test = featureExtraction.ClusteringTendency(BM25_UC_test,tfidf_matrix_uc_test,tfidf_matrix_code_test)
code_CT_BM25_test = featureExtraction.ClusteringTendency(BM25_CC_test,tfidf_matrix_code_test,tfidf_matrix_uc_test)

print("UC_CT_BM25_test",UC_CT_BM25_test.shape)
print("code_CT_BM25_test",code_CT_BM25_test.shape)

np.save('./pickles/UC_CT_BM25_test.npy', UC_CT_BM25_test)
np.save('./pickles/code_CT_BM25_test.npy', code_CT_BM25_test)

# 4.4) Clustering Tendency using JM
UC_CT_JM_test = featureExtraction.ClusteringTendency(JM_UC_test,tfidf_matrix_uc_test,tfidf_matrix_code_test)
code_CT_JM_test = featureExtraction.ClusteringTendency(JM_CC_test,tfidf_matrix_code_test,tfidf_matrix_uc_test)

print("UC_CT_JM_test",UC_CT_JM_test.shape)
print("code_CT_JM_test",code_CT_JM_test.shape)

np.save('./pickles/UC_CT_JM_test.npy', UC_CT_JM_test)
np.save('./pickles/code_CT_JM_test.npy', code_CT_JM_test)

# 4.5) Clustering Tendency using DP
UC_CT_DP_test = featureExtraction.ClusteringTendency(DP_UC_test,tfidf_matrix_uc_test,tfidf_matrix_code_test)
code_CT_DP_test = featureExtraction.ClusteringTendency(DP_CC_test,tfidf_matrix_code_test,tfidf_matrix_uc_test)

print("UC_CT_DP_test",UC_CT_DP_test.shape)
print("code_CT_DP_test",code_CT_DP_test.shape)

np.save('./pickles/UC_CT_DP_test.npy', UC_CT_DP_test)
np.save('./pickles/code_CT_DP_test.npy', code_CT_DP_test)
#20 min

UC_CT_JensenShannon_test (409, 1)
code_CT_JensenShannon_test (174, 1)
UC_CT_VSM_test (409, 1)
code_CT_VSM_test (174, 1)
UC_CT_BM25_test (409, 1)
code_CT_BM25_test (174, 1)
UC_CT_JM_test (409, 1)
code_CT_JM_test (174, 1)
UC_CT_DP_test (409, 1)
code_CT_DP_test (174, 1)


In [20]:
UC_CT_JensenShannon_train = np.load('./pickles/UC_CT_JensenShannon_train.npy')
UC_CT_JensenShannon_train = normalizer.fit_transform(UC_CT_JensenShannon_train)

code_CT_JensenShannon_train = np.load('./pickles/code_CT_JensenShannon_train.npy')
code_CT_JensenShannon_train = normalizer.fit_transform(code_CT_JensenShannon_train)

UC_CT_VSM_train = np.load('./pickles/UC_CT_VSM_train.npy')
UC_CT_VSM_train = normalizer.fit_transform(UC_CT_VSM_train)

code_CT_VSM_train = np.load('./pickles/code_CT_VSM_train.npy')
code_CT_VSM_train = normalizer.fit_transform(code_CT_VSM_train)

UC_CT_BM25_train = np.load('./pickles/UC_CT_BM25_train.npy')
UC_CT_BM25_train = normalizer.fit_transform(UC_CT_BM25_train)

code_CT_BM25_train = np.load('./pickles/code_CT_BM25_train.npy')
code_CT_BM25_train = normalizer.fit_transform(code_CT_BM25_train)

UC_CT_JM_train = np.load('./pickles/UC_CT_JM_train.npy')
UC_CT_JM_train = normalizer.fit_transform(UC_CT_JM_train)

code_CT_JM_train = np.load('./pickles/code_CT_JM_train.npy')
code_CT_JM_train = normalizer.fit_transform(code_CT_JM_train)

UC_CT_DP_train = np.load('./pickles/UC_CT_DP_train.npy')
UC_CT_DP_train = normalizer.fit_transform(UC_CT_DP_train)

code_CT_DP_train = np.load('./pickles/code_CT_DP_train.npy')
code_CT_DP_train = normalizer.fit_transform(code_CT_DP_train)


In [21]:
UC_CT_JensenShannon_test = np.load('./pickles/UC_CT_JensenShannon_test.npy')
UC_CT_JensenShannon_test = normalizer.fit_transform(UC_CT_JensenShannon_test)

code_CT_JensenShannon_test = np.load('./pickles/code_CT_JensenShannon_test.npy')
code_CT_JensenShannon_test = normalizer.fit_transform(code_CT_JensenShannon_test)

UC_CT_VSM_test = np.load('./pickles/UC_CT_VSM_test.npy')
UC_CT_VSM_test = normalizer.fit_transform(UC_CT_VSM_test)

code_CT_VSM_test = np.load('./pickles/code_CT_VSM_test.npy')
code_CT_VSM_test = normalizer.fit_transform(code_CT_VSM_test)

UC_CT_BM25_test = np.load('./pickles/UC_CT_BM25_test.npy')
UC_CT_BM25_test = normalizer.fit_transform(UC_CT_BM25_test)

code_CT_BM25_test = np.load('./pickles/code_CT_BM25_test.npy')
code_CT_BM25_test = normalizer.fit_transform(code_CT_BM25_test)

UC_CT_JM_test = np.load('./pickles/UC_CT_JM_test.npy')
UC_CT_JM_test = normalizer.fit_transform(UC_CT_JM_test)

code_CT_JM_test = np.load('./pickles/code_CT_JM_test.npy')
code_CT_JM_test = normalizer.fit_transform(code_CT_JM_test)

UC_CT_DP_test = np.load('./pickles/UC_CT_DP_test.npy')
UC_CT_DP_test = normalizer.fit_transform(UC_CT_DP_test)

code_CT_DP_test = np.load('./pickles/code_CT_DP_test.npy')
code_CT_DP_test = normalizer.fit_transform(code_CT_DP_test)


In [21]:

#------------------------------Spatial AutoCorrelation  Train-------------------------------------------#
# 5.1) Spatial AutoCorrelation using JensenShannon
UC_SAC_JS_train = featureExtraction.SpatialAutoCorrelation(JS_UC_train.T,tfidf_matrix_code_train)
code_SAC_JS_train = featureExtraction.SpatialAutoCorrelation(JS_CC_train,tfidf_matrix_uc_train)

print("UC_SAC_JS_train",UC_SAC_JS_train.shape)
print("code_SAC_JS_train",code_SAC_JS_train.shape)

np.save('./pickles/UC_SAC_JensenShannon_train.npy',UC_SAC_JS_train)
np.save('./pickles/code_SAC_JensenShannon_train.npy',code_SAC_JS_train)

# 5.2) Spatial AutoCorrelation using VSM
UC_SAC_VSM_train = featureExtraction.SpatialAutoCorrelation(cosine_similarity_UC_train.T,tfidf_matrix_code_train)
code_SAC_VSM_train= featureExtraction.SpatialAutoCorrelation(cosine_similarity_CC_train,tfidf_matrix_uc_train)

print("UC_SAC_VSM_train",UC_SAC_VSM_train.shape)
print("code_SAC_VSM_train",code_SAC_VSM_train.shape)

np.save('./pickles/UC_SAC_VSM_train.npy',UC_SAC_VSM_train)
np.save('./pickles/code_SAC_VSM_train.npy',code_SAC_VSM_train)
# 5.3) Spatial AutoCorrelation using BM25
UC_SAC_BM25_train = featureExtraction.SpatialAutoCorrelation(BM25_UC_train,tfidf_matrix_code_train)
code_SAC_BM25_train = featureExtraction.SpatialAutoCorrelation(BM25_CC_train,tfidf_matrix_uc_train)

print("UC_SAC_BM25_train",UC_SAC_BM25_train.shape)
print("code_SAC_BM25_train",code_SAC_BM25_train.shape)

np.save('./pickles/UC_SAC_BM25_train.npy',UC_SAC_BM25_train)
np.save('./pickles/code_SAC_BM25_train.npy',code_SAC_BM25_train)

# 5.4) Spatial AutoCorrelation using JM
UC_SAC_JM_train = featureExtraction.SpatialAutoCorrelation(JM_UC_train,tfidf_matrix_code_train)
code_SAC_JM_train= featureExtraction.SpatialAutoCorrelation(JM_CC_train,tfidf_matrix_uc_train)

print("UC_SAC_JM_train",UC_SAC_JM_train.shape)
print("code_SAC_JM_train",code_SAC_JM_train.shape)

np.save('./pickles/UC_SAC_JM_train.npy',UC_SAC_JM_train)
np.save('./pickles/code_SAC_JM_train.npy',code_SAC_JM_train)
# 5.5) Spatial AutoCorrelation using DP
UC_SAC_DP_train = featureExtraction.SpatialAutoCorrelation(DP_UC_train,tfidf_matrix_code_train)
code_SAC_DP_train= featureExtraction.SpatialAutoCorrelation(DP_CC_train,tfidf_matrix_uc_train)

print("UC_SAC_DP_train",UC_SAC_DP_train.shape)
print("code_SAC_DP_train",code_SAC_DP_train.shape)

np.save('./pickles/UC_SAC_DP_train.npy',UC_SAC_DP_train)
np.save('./pickles/code_SAC_DP_train.npy',code_SAC_DP_train)
#51.4


UC_SAC_JS_train (2398, 1)
code_SAC_JS_train (1031, 1)
UC_SAC_VSM_train (2398, 1)
code_SAC_VSM_train (1031, 1)
UC_SAC_BM25_train (2398, 1)
code_SAC_BM25_train (1031, 1)
UC_SAC_JM_train (2398, 1)
code_SAC_JM_train (1031, 1)
UC_SAC_DP_train (2398, 1)
code_SAC_DP_train (1031, 1)


In [22]:
#------------------------------ Spatial AutoCorrelation  Test-------------------------------------------#
# 5.1) Spatial AutoCorrelation using JensenShannon
UC_SAC_JS_test = featureExtraction.SpatialAutoCorrelation(JS_UC_test.T,tfidf_matrix_code_test)
code_SAC_JS_test = featureExtraction.SpatialAutoCorrelation(JS_CC_test,tfidf_matrix_uc_test)

print("UC_SAC_JS_test",UC_SAC_JS_test.shape)
print("code_SAC_JS_test",code_SAC_JS_test.shape)

np.save('./pickles/UC_SAC_JensenShannon_test.npy',UC_SAC_JS_test)
np.save('./pickles/code_SAC_JensenShannon_test.npy',code_SAC_JS_test)
# 5.2) Spatial AutoCorrelation using VSM
UC_SAC_VSM_test = featureExtraction.SpatialAutoCorrelation(cosine_similarity_UC_test.T,tfidf_matrix_code_test)
code_SAC_VSM_test= featureExtraction.SpatialAutoCorrelation(cosine_similarity_CC_test,tfidf_matrix_uc_test)

print("UC_SAC_VSM_test",UC_SAC_VSM_test.shape)
print("code_SAC_VSM_test",code_SAC_VSM_test.shape)

np.save('./pickles/UC_SAC_VSM_test.npy',UC_SAC_VSM_test)
np.save('./pickles/code_SAC_VSM_test.npy',code_SAC_VSM_test)

# 5.3) Spatial AutoCorrelation using BM25
UC_SAC_BM25_test = featureExtraction.SpatialAutoCorrelation(BM25_UC_test,tfidf_matrix_code_test)
code_SAC_BM25_test = featureExtraction.SpatialAutoCorrelation(BM25_CC_test,tfidf_matrix_uc_test)

print("UC_SAC_BM25_test",UC_SAC_BM25_test.shape)
print("code_SAC_BM25_test",code_SAC_BM25_test.shape)

np.save('./pickles/UC_SAC_BM25_test.npy',UC_SAC_BM25_test)
np.save('./pickles/code_SAC_BM25_test.npy',code_SAC_BM25_test)

# 5.4) Spatial AutoCorrelation using JM
UC_SAC_JM_test = featureExtraction.SpatialAutoCorrelation(JM_UC_test,tfidf_matrix_code_test)
code_SAC_JM_test= featureExtraction.SpatialAutoCorrelation(JM_CC_test,tfidf_matrix_uc_test)

print("UC_SAC_JM_test",UC_SAC_JM_test.shape)
print("code_SAC_JM_test",code_SAC_JM_test.shape)

np.save('./pickles/UC_SAC_JM_test.npy',UC_SAC_JM_test)
np.save('./pickles/code_SAC_JM_test.npy',code_SAC_JM_test)

# 5.5) Spatial AutoCorrelation using DP
UC_SAC_DP_test = featureExtraction.SpatialAutoCorrelation(DP_UC_test,tfidf_matrix_code_test)
code_SAC_DP_test= featureExtraction.SpatialAutoCorrelation(DP_CC_test,tfidf_matrix_uc_test)

print("UC_SAC_DP_test",UC_SAC_DP_test.shape)
print("code_SAC_DP_test",code_SAC_DP_test.shape)

np.save('./pickles/UC_SAC_DP_test.npy',UC_SAC_DP_test)
np.save('./pickles/code_SAC_DP_test.npy',code_SAC_DP_test)
#7.8sec

UC_SAC_JS_test (409, 1)
code_SAC_JS_test (174, 1)
UC_SAC_VSM_test (409, 1)
code_SAC_VSM_test (174, 1)
UC_SAC_BM25_test (409, 1)
code_SAC_BM25_test (174, 1)
UC_SAC_JM_test (409, 1)
code_SAC_JM_test (174, 1)
UC_SAC_DP_test (409, 1)
code_SAC_DP_test (174, 1)


In [22]:
#------------------------------ Spatial AutoCorrelation Train Loading-------------------------------------------#

UC_SAC_JS_train = np.load('./pickles/UC_SAC_JensenShannon_train.npy')
UC_SAC_JS_train = normalizer.fit_transform(UC_SAC_JS_train)

code_SAC_JS_train = np.load('./pickles/code_SAC_JensenShannon_train.npy')
code_SAC_JS_train = normalizer.fit_transform(code_SAC_JS_train)

UC_SAC_VSM_train = np.load('./pickles/UC_SAC_VSM_train.npy')
UC_SAC_VSM_train = normalizer.fit_transform(UC_SAC_VSM_train)

code_SAC_VSM_train = np.load('./pickles/code_SAC_VSM_train.npy')
code_SAC_VSM_train = normalizer.fit_transform(code_SAC_VSM_train)

UC_SAC_BM25_train = np.load('./pickles/UC_SAC_BM25_train.npy')
UC_SAC_BM25_train = normalizer.fit_transform(UC_SAC_BM25_train)

code_SAC_BM25_train = np.load('./pickles/code_SAC_BM25_train.npy')
code_SAC_BM25_train = normalizer.fit_transform(code_SAC_BM25_train)

UC_SAC_JM_train = np.load('./pickles/UC_SAC_JM_train.npy')
UC_SAC_JM_train = normalizer.fit_transform(UC_SAC_JM_train)

code_SAC_JM_train = np.load('./pickles/code_SAC_JM_train.npy')
code_SAC_JM_train = normalizer.fit_transform(code_SAC_JM_train)

UC_SAC_DP_train = np.load('./pickles/UC_SAC_DP_train.npy')
UC_SAC_DP_train = normalizer.fit_transform(UC_SAC_DP_train)  #ALERT : bytl3 ones ktyera awy , ones w zero

code_SAC_DP_train = np.load('./pickles/code_SAC_DP_train.npy')
code_SAC_DP_train = normalizer.fit_transform(code_SAC_DP_train)


In [23]:
#------------------------------ Spatial AutoCorrelation Test Loading-------------------------------------------#
UC_SAC_JS_test = np.load('./pickles/UC_SAC_JensenShannon_test.npy')
UC_SAC_JS_test = normalizer.fit_transform(UC_SAC_JS_test)

code_SAC_JS_test = np.load('./pickles/code_SAC_JensenShannon_test.npy')
code_SAC_JS_test = normalizer.fit_transform(code_SAC_JS_test)

UC_SAC_VSM_test = np.load('./pickles/UC_SAC_VSM_test.npy')
UC_SAC_VSM_test = normalizer.fit_transform(UC_SAC_VSM_test)

code_SAC_VSM_test = np.load('./pickles/code_SAC_VSM_test.npy')
code_SAC_VSM_test = normalizer.fit_transform(code_SAC_VSM_test)

UC_SAC_BM25_test = np.load('./pickles/UC_SAC_BM25_test.npy')
UC_SAC_BM25_test = normalizer.fit_transform(UC_SAC_BM25_test)

code_SAC_BM25_test = np.load('./pickles/code_SAC_BM25_test.npy')
code_SAC_BM25_test = normalizer.fit_transform(code_SAC_BM25_test)

UC_SAC_JM_test = np.load('./pickles/UC_SAC_JM_test.npy')
UC_SAC_JM_test = normalizer.fit_transform(UC_SAC_JM_test)

code_SAC_JM_test = np.load('./pickles/code_SAC_JM_test.npy')
code_SAC_JM_test = normalizer.fit_transform(code_SAC_JM_test)

UC_SAC_DP_test = np.load('./pickles/UC_SAC_DP_test.npy')
UC_SAC_DP_test = normalizer.fit_transform(UC_SAC_DP_test)

code_SAC_DP_test = np.load('./pickles/code_SAC_DP_test.npy')
code_SAC_DP_test = normalizer.fit_transform(code_SAC_DP_test)


In [25]:
#----------------------WeightedInformationGain Train and Test-----------------#
# 6.1) WIG using JensenShannon

UC_WIG_score_JensenShannon_train = featureExtraction.WeightedInformationGain(UC_documents_train,code_documents_train,JS_UC_train.T,tf_code_dict_train,np.sum(list(tf_code_dict_train.values())))
code_WIG_score_JensenShannon_train = featureExtraction.WeightedInformationGain(code_documents_train,UC_documents_train,JS_CC_train,tf_uc_dict_train,np.sum(list(tf_uc_dict_train.values())))

np.save('./pickles/UC_WIG_score_JensenShannon_train.npy',UC_WIG_score_JensenShannon_train)
np.save('./pickles/code_WIG_score_JensenShannon_train.npy',code_WIG_score_JensenShannon_train)

UC_WIG_score_JensenShannon_test = featureExtraction.WeightedInformationGain(UC_documents_test,code_documents_test,JS_UC_test.T,tf_code_dict_test,np.sum(list(tf_code_dict_test.values())))
code_WIG_score_JensenShannon_test = featureExtraction.WeightedInformationGain(code_documents_test,UC_documents_test,JS_CC_test,tf_uc_dict_test,np.sum(list(tf_uc_dict_test.values())))

np.save('./pickles/UC_WIG_score_JensenShannon_test.npy',UC_WIG_score_JensenShannon_test)
np.save('./pickles/code_WIG_score_JensenShannon_test.npy',code_WIG_score_JensenShannon_test)

# 6.2) WIG Score using VSM

UC_WIG_score_VSM_train = featureExtraction.WeightedInformationGain(UC_documents_train,code_documents_train,cosine_similarity_UC_train.T,tf_code_dict_train,np.sum(list(tf_code_dict_train.values())))
code_WIG_score_VSM_train = featureExtraction.WeightedInformationGain(code_documents_train,UC_documents_train,cosine_similarity_CC_train,tf_uc_dict_train,np.sum(list(tf_uc_dict_train.values())))

np.save('./pickles/UC_WIG_score_VSM_train.npy',UC_WIG_score_VSM_train)
np.save('./pickles/code_WIG_score_VSM_train.npy',code_WIG_score_VSM_train)

UC_WIG_score_VSM_test = featureExtraction.WeightedInformationGain(UC_documents_test,code_documents_test,cosine_similarity_UC_test.T,tf_code_dict_test,np.sum(list(tf_code_dict_test.values())))
code_WIG_score_VSM_test = featureExtraction.WeightedInformationGain(code_documents_test,UC_documents_test,cosine_similarity_CC_test,tf_uc_dict_test,np.sum(list(tf_uc_dict_test.values())))

np.save('./pickles/UC_WIG_score_VSM_test.npy',UC_WIG_score_VSM_test)
np.save('./pickles/code_WIG_score_VSM_test.npy',code_WIG_score_VSM_test)

#6.3) WIG Score using BM25

UC_WIG_score_BM25_train = featureExtraction.WeightedInformationGain(UC_documents_train,code_documents_train,BM25_UC_train,tf_code_dict_train,np.sum(list(tf_code_dict_train.values())))
code_WIG_score_BM25_train = featureExtraction.WeightedInformationGain(code_documents_train,UC_documents_train,BM25_CC_train,tf_uc_dict_train,np.sum(list(tf_uc_dict_train.values())))

np.save('./pickles/UC_WIG_score_BM25_train.npy',UC_WIG_score_BM25_train)
np.save('./pickles/code_WIG_score_BM25_train.npy',code_WIG_score_BM25_train)


UC_WIG_score_BM25_test = featureExtraction.WeightedInformationGain(UC_documents_test,code_documents_test,BM25_UC_test,tf_code_dict_test,np.sum(list(tf_code_dict_test.values())))
code_WIG_score_BM25_test = featureExtraction.WeightedInformationGain(code_documents_test,UC_documents_test,BM25_CC_test,tf_uc_dict_test,np.sum(list(tf_uc_dict_test.values())))

np.save('./pickles/UC_WIG_score_BM25_test.npy',UC_WIG_score_BM25_test)
np.save('./pickles/code_WIG_score_BM25_test.npy',code_WIG_score_BM25_test)

#6.4) WIG Score using JM

UC_WIG_score_JM_train = featureExtraction.WeightedInformationGain(UC_documents_train,code_documents_train,JM_UC_train,tf_code_dict_train,np.sum(list(tf_code_dict_train.values())))
code_WIG_score_JM_train = featureExtraction.WeightedInformationGain(code_documents_train,UC_documents_train,JM_CC_train,tf_uc_dict_train,np.sum(list(tf_uc_dict_train.values())))

np.save('./pickles/UC_WIG_score_JM_train.npy',UC_WIG_score_JM_train)
np.save('./pickles/code_WIG_score_JM_train.npy',code_WIG_score_JM_train)

UC_WIG_score_JM_test = featureExtraction.WeightedInformationGain(UC_documents_test,code_documents_test,JM_UC_test,tf_code_dict_test,np.sum(list(tf_code_dict_test.values())))
code_WIG_score_JM_test = featureExtraction.WeightedInformationGain(code_documents_test,UC_documents_test,JM_CC_test,tf_uc_dict_test,np.sum(list(tf_uc_dict_test.values())))

np.save('./pickles/UC_WIG_score_JM_test.npy',UC_WIG_score_JM_test)
np.save('./pickles/code_WIG_score_JM_test.npy',code_WIG_score_JM_test)

#6.5) WIG Score using DP

UC_WIG_score_DP_train = featureExtraction.WeightedInformationGain(UC_documents_train,code_documents_train,DP_UC_train,tf_code_dict_train,np.sum(list(tf_code_dict_train.values())))
code_WIG_score_DP_train = featureExtraction.WeightedInformationGain(code_documents_train,UC_documents_train,DP_CC_train,tf_uc_dict_train,np.sum(list(tf_uc_dict_train.values())))

np.save('./pickles/UC_WIG_score_DP_train.npy',UC_WIG_score_DP_train)
np.save('./pickles/code_WIG_score_DP_train.npy',code_WIG_score_DP_train)

UC_WIG_score_DP_test = featureExtraction.WeightedInformationGain(UC_documents_test,code_documents_test,DP_UC_test,tf_code_dict_test,np.sum(list(tf_code_dict_test.values())))
code_WIG_score_DP_test = featureExtraction.WeightedInformationGain(code_documents_test,UC_documents_test,DP_CC_test,tf_uc_dict_test,np.sum(list(tf_uc_dict_test.values())))

np.save('./pickles/UC_WIG_score_DP_test.npy',UC_WIG_score_DP_test)
np.save('./pickles/code_WIG_score_DP_test.npy',code_WIG_score_DP_test)

In [24]:
UC_WIG_score_JensenShannon_train = np.load('./pickles/UC_WIG_score_JensenShannon_train.npy')
UC_WIG_score_JensenShannon_train = normalizer.fit_transform(UC_WIG_score_JensenShannon_train)

code_WIG_score_JensenShannon_train = np.load('./pickles/code_WIG_score_JensenShannon_train.npy') #ALERT : el majority rakam whed
code_WIG_score_JensenShannon_train = normalizer.fit_transform(code_WIG_score_JensenShannon_train)

UC_WIG_score_JensenShannon_test = np.load('./pickles/UC_WIG_score_JensenShannon_test.npy')
UC_WIG_score_JensenShannon_test = normalizer.fit_transform(UC_WIG_score_JensenShannon_test)

code_WIG_score_JensenShannon_test = np.load('./pickles/code_WIG_score_JensenShannon_test.npy') #ALERT : el majority rakam whed
code_WIG_score_JensenShannon_test = normalizer.fit_transform(code_WIG_score_JensenShannon_test)

UC_WIG_score_VSM_train = np.load('./pickles/UC_WIG_score_VSM_train.npy')
UC_WIG_score_VSM_train = normalizer.fit_transform(UC_WIG_score_VSM_train)

code_WIG_score_VSM_train = np.load('./pickles/code_WIG_score_VSM_train.npy')
code_WIG_score_VSM_train = normalizer.fit_transform(code_WIG_score_VSM_train)  # ALERT : zeros kytyera awy

UC_WIG_score_VSM_test = np.load('./pickles/UC_WIG_score_VSM_test.npy')
UC_WIG_score_VSM_test = normalizer.fit_transform(UC_WIG_score_VSM_test)

code_WIG_score_VSM_test = np.load('./pickles/code_WIG_score_VSM_test.npy')
code_WIG_score_VSM_test = normalizer.fit_transform(code_WIG_score_VSM_test) # ALERT : zeros kytyera awy

UC_WIG_score_BM25_train = np.load('./pickles/UC_WIG_score_BM25_train.npy')
UC_WIG_score_BM25_train = normalizer.fit_transform(UC_WIG_score_BM25_train)

code_WIG_score_BM25_train = np.load('./pickles/code_WIG_score_BM25_train.npy')
code_WIG_score_BM25_train = normalizer.fit_transform(code_WIG_score_BM25_train) # ALERT : zeros kytyera awy
 
UC_WIG_score_BM25_test = np.load('./pickles/UC_WIG_score_BM25_test.npy')
UC_WIG_score_BM25_test = normalizer.fit_transform(UC_WIG_score_BM25_test)

code_WIG_score_BM25_test = np.load('./pickles/code_WIG_score_BM25_test.npy')
code_WIG_score_BM25_test = normalizer.fit_transform(code_WIG_score_BM25_test)#ALERT : el majority rakam whed

UC_WIG_score_JM_train = np.load('./pickles/UC_WIG_score_JM_train.npy')
UC_WIG_score_JM_train = normalizer.fit_transform(UC_WIG_score_JM_train)

code_WIG_score_JM_train = np.load('./pickles/code_WIG_score_JM_train.npy')
code_WIG_score_JM_train = normalizer.fit_transform(code_WIG_score_JM_train) # ALERT : zeros kytyera awy

UC_WIG_score_JM_test = np.load('./pickles/UC_WIG_score_JM_test.npy')
UC_WIG_score_JM_test = normalizer.fit_transform(UC_WIG_score_JM_test)

code_WIG_score_JM_test = np.load('./pickles/code_WIG_score_JM_test.npy')
code_WIG_score_JM_test = normalizer.fit_transform(code_WIG_score_JM_test) # ALERT : zeros kytyera awy

UC_WIG_score_DP_train = np.load('./pickles/UC_WIG_score_DP_train.npy')
UC_WIG_score_DP_train = normalizer.fit_transform(UC_WIG_score_DP_train)

code_WIG_score_DP_train = np.load('./pickles/code_WIG_score_DP_train.npy')#ALERT : el majority rakam whed
code_WIG_score_DP_train = normalizer.fit_transform(code_WIG_score_DP_train)

UC_WIG_score_DP_test = np.load('./pickles/UC_WIG_score_DP_test.npy')
UC_WIG_score_DP_test = normalizer.fit_transform(UC_WIG_score_DP_test)

code_WIG_score_DP_test = np.load('./pickles/code_WIG_score_DP_test.npy')#ALERT : el majority rakam whed
code_WIG_score_DP_test = normalizer.fit_transform(code_WIG_score_DP_test)


In [27]:
#--------------------NormalizedQueryCommitment Train and Test -----------#
# 7.1) NQC using JensenShannon
UC_NQC_JensenShannon_train = featureExtraction.NormalizedQueryCommitment(JS_UC_train.T)
code_NQC_JensenShannon_train = featureExtraction.NormalizedQueryCommitment(JS_CC_train)

np.save('./pickles/UC_NQC_JensenShannon_train.npy',UC_NQC_JensenShannon_train)
np.save('./pickles/code_NQC_JensenShannon_train.npy',code_NQC_JensenShannon_train)

UC_NQC_JensenShannon_test = featureExtraction.NormalizedQueryCommitment(JS_UC_test.T)
code_NQC_JensenShannon_test = featureExtraction.NormalizedQueryCommitment(JS_CC_test)

np.save('./pickles/UC_NQC_JensenShannon_test.npy',UC_NQC_JensenShannon_test)
np.save('./pickles/code_NQC_JensenShannon_test.npy',code_NQC_JensenShannon_test)

# 7.2) NQC using VSM
UC_NQC_VSM_train = featureExtraction.NormalizedQueryCommitment(cosine_similarity_UC_train.T)
code_NQC_VSM_train = featureExtraction.NormalizedQueryCommitment(cosine_similarity_CC_train)

np.save('./pickles/UC_NQC_VSM_train.npy',UC_NQC_VSM_train)
np.save('./pickles/code_NQC_VSM_train.npy',code_NQC_VSM_train)

UC_NQC_VSM_test = featureExtraction.NormalizedQueryCommitment(cosine_similarity_UC_test.T)
code_NQC_VSM_test = featureExtraction.NormalizedQueryCommitment(cosine_similarity_CC_test)

np.save('./pickles/UC_NQC_VSM_test.npy',UC_NQC_VSM_test)
np.save('./pickles/code_NQC_VSM_test.npy',code_NQC_VSM_test)

# 7.3) NQC using BM25
UC_NQC_BM25_train = featureExtraction.NormalizedQueryCommitment(BM25_UC_train)
code_NQC_BM25_train = featureExtraction.NormalizedQueryCommitment(BM25_CC_train)

np.save('./pickles/UC_NQC_BM25_train.npy',UC_NQC_BM25_train)
np.save('./pickles/code_NQC_BM25_train.npy',code_NQC_BM25_train)

UC_NQC_BM25_test = featureExtraction.NormalizedQueryCommitment(BM25_UC_test)
code_NQC_BM25_test = featureExtraction.NormalizedQueryCommitment(BM25_CC_test)

np.save('./pickles/UC_NQC_BM25_test.npy',UC_NQC_BM25_test)
np.save('./pickles/code_NQC_BM25_test.npy',code_NQC_BM25_test)

# 7.4) NQC using JM
UC_NQC_JM_train = featureExtraction.NormalizedQueryCommitment(JM_UC_train)
code_NQC_JM_train = featureExtraction.NormalizedQueryCommitment(JM_CC_train)

np.save('./pickles/UC_NQC_JM_train.npy',UC_NQC_JM_train)
np.save('./pickles/code_NQC_JM_train.npy',code_NQC_JM_train)

UC_NQC_JM_test = featureExtraction.NormalizedQueryCommitment(JM_UC_test)
code_NQC_JM_test = featureExtraction.NormalizedQueryCommitment(JM_CC_test)

np.save('./pickles/UC_NQC_JM_test.npy',UC_NQC_JM_test)
np.save('./pickles/code_NQC_JM_test.npy',code_NQC_JM_test)

# 7.5) NQC using DP
UC_NQC_DP_train = featureExtraction.NormalizedQueryCommitment(DP_UC_train)
code_NQC_DP_train = featureExtraction.NormalizedQueryCommitment(DP_CC_train)

np.save('./pickles/UC_NQC_DP_train.npy',UC_NQC_DP_train)
np.save('./pickles/code_NQC_DP_train.npy',code_NQC_DP_train)

UC_NQC_DP_test = featureExtraction.NormalizedQueryCommitment(DP_UC_test)
code_NQC_DP_test = featureExtraction.NormalizedQueryCommitment(DP_CC_test)

np.save('./pickles/UC_NQC_DP_test.npy',UC_NQC_DP_test)
np.save('./pickles/code_NQC_DP_test.npy',code_NQC_DP_test)

In [37]:
UC_NQC_JensenShannon_train = np.load('./pickles/UC_NQC_JensenShannon_train.npy')
UC_NQC_JensenShannon_train = normalizer.fit_transform(UC_NQC_JensenShannon_train)

code_NQC_JensenShannon_train = np.load('./pickles/code_NQC_JensenShannon_train.npy')
code_NQC_JensenShannon_train = normalizer.fit_transform(code_NQC_JensenShannon_train)

UC_NQC_JensenShannon_test = np.load('./pickles/UC_NQC_JensenShannon_test.npy')
UC_NQC_JensenShannon_test = normalizer.fit_transform(UC_NQC_JensenShannon_test)

code_NQC_JensenShannon_test = np.load('./pickles/code_NQC_JensenShannon_test.npy')
code_NQC_JensenShannon_test = normalizer.fit_transform(code_NQC_JensenShannon_test)

UC_NQC_VSM_train = np.load('./pickles/UC_NQC_VSM_train.npy')
UC_NQC_VSM_train = normalizer.fit_transform(UC_NQC_VSM_train)

code_NQC_VSM_train = np.load('./pickles/code_NQC_VSM_train.npy')
code_NQC_VSM_train = normalizer.fit_transform(code_NQC_VSM_train)

UC_NQC_VSM_test = np.load('./pickles/UC_NQC_VSM_test.npy')
UC_NQC_VSM_test = normalizer.fit_transform(UC_NQC_VSM_test)

code_NQC_VSM_test = np.load('./pickles/code_NQC_VSM_test.npy')
code_NQC_VSM_test = normalizer.fit_transform(code_NQC_VSM_test)

UC_NQC_BM25_train = np.load('./pickles/UC_NQC_BM25_train.npy')
UC_NQC_BM25_train = normalizer.fit_transform(UC_NQC_BM25_train)

code_NQC_BM25_train = np.load('./pickles/code_NQC_BM25_train.npy')
code_NQC_BM25_train = normalizer.fit_transform(code_NQC_BM25_train)

UC_NQC_BM25_test = np.load('./pickles/UC_NQC_BM25_test.npy')
UC_NQC_BM25_test = normalizer.fit_transform(UC_NQC_BM25_test)

code_NQC_BM25_test = np.load('./pickles/code_NQC_BM25_test.npy')
code_NQC_BM25_test = normalizer.fit_transform(code_NQC_BM25_test)

UC_NQC_JM_train = np.load('./pickles/UC_NQC_JM_train.npy')
UC_NQC_JM_train = normalizer.fit_transform(UC_NQC_JM_train)

code_NQC_JM_train = np.load('./pickles/code_NQC_JM_train.npy')
code_NQC_JM_train = normalizer.fit_transform(code_NQC_JM_train)

UC_NQC_JM_test = np.load('./pickles/UC_NQC_JM_test.npy')
UC_NQC_JM_test = normalizer.fit_transform(UC_NQC_JM_test)

code_NQC_JM_test = np.load('./pickles/code_NQC_JM_test.npy')
code_NQC_JM_test = normalizer.fit_transform(code_NQC_JM_test)

UC_NQC_DP_train = np.load('./pickles/UC_NQC_DP_train.npy')
UC_NQC_DP_train = normalizer.fit_transform(UC_NQC_DP_train)

code_NQC_DP_train = np.load('./pickles/code_NQC_DP_train.npy')
code_NQC_DP_train = normalizer.fit_transform(code_NQC_DP_train)

UC_NQC_DP_test = np.load('./pickles/UC_NQC_DP_test.npy')
UC_NQC_DP_test = normalizer.fit_transform(UC_NQC_DP_test)

code_NQC_DP_test = np.load('./pickles/code_NQC_DP_test.npy')
code_NQC_DP_test = normalizer.fit_transform(code_NQC_DP_test)

### Document Statistics Features

In [29]:
from FeatureExtraction import *
featureExtraction = FeatureExtraction(Vocab_UC_train)
num_terms_code_train, num_terms_UC_train, num_unique_terms_code_train, num_unique_terms_UC_train, num_overlapping_terms_train = featureExtraction.DocumentStatistics(UC_documents_train, code_documents_train)
#18min

In [30]:
featureExtraction = FeatureExtraction(Vocab_UC_train)
num_terms_code_test, num_terms_UC_test, num_unique_terms_code_test, num_unique_terms_UC_test, num_overlapping_terms_test = featureExtraction.DocumentStatistics(UC_documents_test, code_documents_test)

In [31]:
np.save('./pickles/num_terms_code_train.npy',num_terms_code_train)
np.save('./pickles/num_terms_UC_train.npy',num_terms_UC_train)
np.save('./pickles/num_unique_terms_code_train.npy',num_unique_terms_code_train)
np.save('./pickles/num_unique_terms_UC_train.npy',num_unique_terms_UC_train)
np.save('./pickles/num_overlapping_terms_train.npy',num_overlapping_terms_train)

In [32]:
np.save('./pickles/num_terms_code_test.npy',num_terms_code_test)
np.save('./pickles/num_terms_UC_test.npy',num_terms_UC_test)
np.save('./pickles/num_unique_terms_code_test.npy',num_unique_terms_code_test)
np.save('./pickles/num_unique_terms_UC_test.npy',num_unique_terms_UC_test)
np.save('./pickles/num_overlapping_terms_test.npy',num_overlapping_terms_test)

In [26]:
num_terms_code_train = np.load('./pickles/num_terms_code_train.npy')
num_terms_UC_train = np.load('./pickles/num_terms_UC_train.npy')
num_unique_terms_code_train = np.load('./pickles/num_unique_terms_code_train.npy')
num_unique_terms_UC_train =np.load('./pickles/num_unique_terms_UC_train.npy')
num_overlapping_terms_train = np.load('./pickles/num_overlapping_terms_train.npy')

In [27]:
num_terms_code_test = np.load('./pickles/num_terms_code_test.npy')
num_terms_UC_test= np.load('./pickles/num_terms_UC_test.npy')
num_unique_terms_code_test = np.load('./pickles/num_unique_terms_code_test.npy')
num_unique_terms_UC_test =np.load('./pickles/num_unique_terms_UC_test.npy')
num_overlapping_terms_test = np.load('./pickles/num_overlapping_terms_test.npy')

### Tiling and stacking the 126 feature

In [28]:
avg_idf_uc_train = np.array(avg_idf_uc_train)
avg_idf_code_train = np.array(avg_idf_code_train)
max_idf_uc_train = np.array(max_idf_uc_train)
max_idf_code_train = np.array(max_idf_code_train)
dev_idf_uc_train = np.array(dev_idf_uc_train)
dev_idf_code_train = np.array(dev_idf_code_train)
avg_ictf_uc_train = np.array(avg_ictf_uc_train)
avg_ictf_code_train = np.array(avg_ictf_code_train)
max_ictf_uc_train = np.array(max_ictf_uc_train)
max_ictf_code_train = np.array(max_ictf_code_train)
dev_ictf_uc_train = np.array(dev_ictf_uc_train)
dev_ictf_code_train = np.array(dev_ictf_code_train)
avg_entropy_uc_train = np.array(avg_entropy_uc_train)
avg_entropy_code_train = np.array(avg_entropy_code_train)
max_entropy_uc_train = np.array(max_entropy_uc_train)
max_entropy_code_train = np.array(max_entropy_code_train)
med_entropy_uc_train = np.array(med_entropy_uc_train)
med_entropy_code_train = np.array(med_entropy_code_train)
dev_entropy_uc_train = np.array(dev_entropy_uc_train)
dev_entropy_code_train = np.array(dev_entropy_code_train)
avg_variance_uc_train = np.array(avg_variance_uc_train)
avg_variance_code_train = np.array(avg_variance_code_train)
max_variance_uc_train = np.array(max_variance_uc_train)
max_variance_code_train = np.array(max_variance_code_train)
sum_variance_uc_train = np.array(sum_variance_uc_train)
sum_variance_code_train = np.array(sum_variance_code_train)
avg_scq_uc_train = np.array(avg_scq_uc_train)
avg_scq_code_train = np.array(avg_scq_code_train)
max_scq_uc_train = np.array(max_scq_uc_train)
max_scq_code_train = np.array(max_scq_code_train)
sum_sqc_uc_train = np.array(sum_sqc_uc_train)
sum_sqc_code_train = np.array(sum_sqc_code_train)
avg_pmi_uc_train = np.array(avg_pmi_uc_train)
avg_pmi_code_train = np.array(avg_pmi_code_train)
max_pmi_uc_train = np.array(max_pmi_uc_train)
max_pmi_code_train = np.array(max_pmi_code_train)
qs_uc_train = np.array(qs_uc_train)
qs_code_train = np.array(qs_code_train)
UC_SCS_train = np.array(UC_SCS_train)
CC_SCS_train = np.array(CC_SCS_train)
UC_CoherenceScore_train = np.array(UC_CoherenceScore_train)
CC_CoherenceScore_train = np.array(CC_CoherenceScore_train)

In [29]:
avg_idf_uc_train_reshaped = np.tile(avg_idf_uc_train, (1, avg_idf_code_train.shape[0]))
avg_idf_code_train_reshaped = np.tile(avg_idf_code_train, (1, avg_idf_uc_train.shape[0]))

max_idf_uc_train_reshaped = np.tile(max_idf_uc_train, (1, max_idf_code_train.shape[0]))
max_idf_code_train_reshaped = np.tile(max_idf_code_train, (1, max_idf_uc_train.shape[0]))

dev_idf_uc_train_reshaped = np.tile(dev_idf_uc_train, (1, dev_idf_code_train.shape[0]))
dev_idf_code_train_reshaped = np.tile(dev_idf_code_train, (1, dev_idf_uc_train.shape[0]))

avg_ictf_uc_train_reshaped = np.tile(avg_ictf_uc_train, (1, avg_ictf_code_train.shape[0]))
avg_ictf_code_train_reshaped = np.tile(avg_ictf_code_train, (1, avg_ictf_uc_train.shape[0]))

max_ictf_uc_train_reshaped = np.tile(max_ictf_uc_train, (1, max_ictf_code_train.shape[0]))
max_ictf_code_train_reshaped = np.tile(max_ictf_code_train, (1, max_ictf_uc_train.shape[0]))

dev_ictf_uc_train_reshaped = np.tile(dev_ictf_uc_train, (1, dev_ictf_code_train.shape[0]))
dev_ictf_code_train_reshaped = np.tile(dev_ictf_code_train, (1, dev_ictf_uc_train.shape[0]))

avg_entropy_uc_train_reshaped = np.tile(avg_entropy_uc_train, (1, avg_entropy_code_train.shape[0]))
avg_entropy_code_train_reshaped = np.tile(avg_entropy_code_train, (1, avg_entropy_uc_train.shape[0]))

max_entropy_uc_train_reshaped = np.tile(max_entropy_uc_train, (1, max_entropy_code_train.shape[0]))
max_entropy_code_train_reshaped = np.tile(max_entropy_code_train, (1, max_entropy_uc_train.shape[0]))

med_entropy_uc_train_reshaped = np.tile(med_entropy_uc_train, (1, med_entropy_code_train.shape[0]))
med_entropy_code_train_reshaped = np.tile(med_entropy_code_train, (1, med_entropy_uc_train.shape[0]))

dev_entropy_uc_train_reshaped = np.tile(dev_entropy_uc_train, (1, dev_entropy_code_train.shape[0]))
dev_entropy_code_train_reshaped = np.tile(dev_entropy_code_train, (1, dev_entropy_uc_train.shape[0]))

avg_variance_uc_train_reshaped = np.tile(avg_variance_uc_train, (1, avg_variance_code_train.shape[0]))
avg_variance_code_train_reshaped = np.tile(avg_variance_code_train, (1, avg_variance_uc_train.shape[0]))

max_variance_uc_train_reshaped = np.tile(max_variance_uc_train, (1, max_variance_code_train.shape[0]))
max_variance_code_train_reshaped = np.tile(max_variance_code_train, (1, max_variance_uc_train.shape[0]))

sum_variance_uc_train_reshaped = np.tile(sum_variance_uc_train, (1, sum_variance_code_train.shape[0]))
sum_variance_code_train_reshaped = np.tile(sum_variance_code_train, (1, sum_variance_uc_train.shape[0]))

avg_scq_uc_train_reshaped = np.tile(avg_scq_uc_train, (1, avg_scq_code_train.shape[0]))
avg_scq_code_train_reshaped = np.tile(avg_scq_code_train, (1, avg_scq_uc_train.shape[0]))

max_scq_uc_train_reshaped = np.tile(max_scq_uc_train, (1, max_scq_code_train.shape[0]))
max_scq_code_train_reshaped = np.tile(max_scq_code_train, (1, max_scq_uc_train.shape[0]))

sum_sqc_uc_train_reshaped = np.tile(sum_sqc_uc_train, (1, sum_sqc_code_train.shape[0]))
sum_sqc_code_train_reshaped = np.tile(sum_sqc_code_train, (1, sum_sqc_uc_train.shape[0]))

avg_pmi_uc_train_reshaped = np.tile(avg_pmi_uc_train, (1, avg_pmi_code_train.shape[0]))
avg_pmi_code_train_reshaped = np.tile(avg_pmi_code_train, (1, avg_pmi_uc_train.shape[0]))

max_pmi_uc_train_reshaped = np.tile(max_pmi_uc_train, (1, max_pmi_code_train.shape[0]))
max_pmi_code_train_reshaped = np.tile(max_pmi_code_train, (1, max_pmi_uc_train.shape[0]))

qs_uc_train_reshaped = np.tile(qs_uc_train, (1, qs_code_train.shape[0]))
qs_code_train_reshaped = np.tile(qs_code_train, (1, qs_uc_train.shape[0]))

UC_SCS_train_reshaped = np.tile(UC_SCS_train, (1, CC_SCS_train.shape[0]))
CC_SCS_train_reshaped = np.tile(CC_SCS_train, (1, UC_SCS_train.shape[0]))

UC_CoherenceScore_train_reshaped = np.tile(UC_CoherenceScore_train, (1, CC_CoherenceScore_train.shape[0]))
CC_CoherenceScore_train_reshaped = np.tile(CC_CoherenceScore_train, (1, UC_CoherenceScore_train.shape[0]))

In [30]:
UC_RS_JS_train_reshaped = np.tile(UC_RS_JS_train, (1, code_RS_JS_train.shape[0]))
code_RS_JS_train_reshaped = np.tile(code_RS_JS_train, (1, UC_RS_JS_train.shape[0]))

UC_FRC_JS_train_reshaped = np.tile(UC_FRC_JS_train, (1, code_FRC_JS_train.shape[0]))
code_FRC_JS_train_reshaped = np.tile(code_FRC_JS_train, (1, UC_FRC_JS_train.shape[0]))

UC_RS_VSM_train_reshaped = np.tile(UC_RS_VSM_train, (1, code_RS_VSM_train.shape[0]))
code_RS_VSM_train_reshaped = np.tile(code_RS_VSM_train, (1, UC_RS_VSM_train.shape[0]))

UC_FRC_VSM_train_reshaped = np.tile(UC_FRC_VSM_train, (1, code_FRC_VSM_train.shape[0]))
code_FRC_VSM_train_reshaped = np.tile(code_FRC_VSM_train, (1, UC_FRC_VSM_train.shape[0]))

UC_RS_BM25_train_reshaped = np.tile(UC_RS_BM25_train, (1, code_RS_BM25_train.shape[0]))
code_RS_BM25_train_reshaped = np.tile(code_RS_BM25_train, (1, UC_RS_BM25_train.shape[0]))

UC_FRC_BM25_train_reshaped = np.tile(UC_FRC_BM25_train, (1, code_FRC_BM25_train.shape[0]))
code_FRC_BM25_train_reshaped = np.tile(code_FRC_BM25_train, (1, UC_FRC_BM25_train.shape[0]))

UC_RS_JM_train_reshaped = np.tile(UC_RS_JM_train, (1, code_RS_JM_train.shape[0]))
code_RS_JM_train_reshaped = np.tile(code_RS_JM_train, (1, UC_RS_JM_train.shape[0]))

UC_FRC_JM_train_reshaped = np.tile(UC_FRC_JM_train, (1, code_FRC_JM_train.shape[0]))
code_FRC_JM_train_reshaped = np.tile(code_FRC_JM_train, (1, UC_FRC_JM_train.shape[0]))

UC_RS_DP_train_reshaped = np.tile(UC_RS_DP_train, (1, code_RS_DP_train.shape[0]))
code_RS_DP_train_reshaped = np.tile(code_RS_DP_train, (1, UC_RS_DP_train.shape[0]))

UC_FRC_DP_train_reshaped = np.tile(UC_FRC_DP_train, (1, code_FRC_DP_train.shape[0]))
code_FRC_DP_train_reshaped = np.tile(code_FRC_DP_train, (1, UC_FRC_DP_train.shape[0]))

In [31]:
UC_CT_JensenShannon_train_reshaped = np.tile(UC_CT_JensenShannon_train, (1, code_CT_JensenShannon_train.shape[0]))
code_CT_JensenShannon_train_reshaped = np.tile(code_CT_JensenShannon_train, (1, UC_CT_JensenShannon_train.shape[0]))

UC_CT_VSM_train_reshaped = np.tile(UC_CT_VSM_train, (1, code_CT_VSM_train.shape[0]))
code_CT_VSM_train_reshaped = np.tile(code_CT_VSM_train, (1, UC_CT_VSM_train.shape[0]))

UC_CT_BM25_train_reshaped = np.tile(UC_CT_BM25_train, (1, code_CT_BM25_train.shape[0]))
code_CT_BM25_train_reshaped = np.tile(code_CT_BM25_train, (1, UC_CT_BM25_train.shape[0]))

UC_CT_JM_train_reshaped = np.tile(UC_CT_JM_train, (1, code_CT_JM_train.shape[0]))
code_CT_JM_train_reshaped = np.tile(code_CT_JM_train, (1, UC_CT_JM_train.shape[0]))

UC_CT_DP_train_reshaped = np.tile(UC_CT_DP_train, (1, code_CT_DP_train.shape[0]))
code_CT_DP_train_reshaped = np.tile(code_CT_DP_train, (1, UC_CT_DP_train.shape[0]))

In [32]:
UC_SAC_JS_train_reshaped = np.tile(UC_SAC_JS_train, (1, code_SAC_JS_train.shape[0]))
code_SAC_JS_train_reshaped = np.tile(code_SAC_JS_train, (1, UC_SAC_JS_train.shape[0]))

UC_SAC_VSM_train_reshaped = np.tile(UC_SAC_VSM_train, (1, code_SAC_VSM_train.shape[0]))
code_SAC_VSM_train_reshaped = np.tile(code_SAC_VSM_train, (1, UC_SAC_VSM_train.shape[0]))

UC_SAC_BM25_train_reshaped = np.tile(UC_SAC_BM25_train, (1, code_SAC_BM25_train.shape[0]))
code_SAC_BM25_train_reshaped = np.tile(code_SAC_BM25_train, (1, UC_SAC_BM25_train.shape[0]))

UC_SAC_JM_train_reshaped = np.tile(UC_SAC_JM_train, (1, code_SAC_JM_train.shape[0]))
code_SAC_JM_train_reshaped = np.tile(code_SAC_JM_train, (1, UC_SAC_JM_train.shape[0]))

UC_SAC_DP_train_reshaped = np.tile(UC_SAC_DP_train, (1, code_SAC_DP_train.shape[0]))
code_SAC_DP_train_reshaped = np.tile(code_SAC_DP_train, (1, UC_SAC_DP_train.shape[0]))

print(code_SAC_DP_train.shape)

(1031, 1)


In [33]:
UC_WIG_score_JensenShannon_train_reshaped = np.tile(UC_WIG_score_JensenShannon_train, (1, code_WIG_score_JensenShannon_train.shape[0]))
code_WIG_score_JensenShannon_train_reshaped = np.tile(code_WIG_score_JensenShannon_train, (1, UC_WIG_score_JensenShannon_train.shape[0]))

UC_WIG_score_VSM_train_reshaped = np.tile(UC_WIG_score_VSM_train, (1, code_WIG_score_VSM_train.shape[0]))
code_WIG_score_VSM_train_reshaped = np.tile(code_WIG_score_VSM_train, (1, UC_WIG_score_VSM_train.shape[0]))

UC_WIG_score_BM25_train_reshaped = np.tile(UC_WIG_score_BM25_train, (1, code_WIG_score_BM25_train.shape[0]))
code_WIG_score_BM25_train_reshaped = np.tile(code_WIG_score_BM25_train, (1, UC_WIG_score_BM25_train.shape[0]))

UC_WIG_score_JM_train_reshaped = np.tile(UC_WIG_score_JM_train, (1, code_WIG_score_JM_train.shape[0]))
code_WIG_score_JM_train_reshaped = np.tile(code_WIG_score_JM_train, (1, UC_WIG_score_JM_train.shape[0]))

UC_WIG_score_DP_train_reshaped = np.tile(UC_WIG_score_DP_train, (1, code_WIG_score_DP_train.shape[0]))
code_WIG_score_DP_train_reshaped = np.tile(code_WIG_score_DP_train, (1, UC_WIG_score_DP_train.shape[0]))

In [38]:
UC_NQC_JensenShannon_train_reshaped = np.tile(UC_NQC_JensenShannon_train, (1, code_NQC_JensenShannon_train.shape[0]))
code_NQC_JensenShannon_train_reshaped = np.tile(code_NQC_JensenShannon_train, (1, UC_NQC_JensenShannon_train.shape[0]))


UC_NQC_VSM_train_reshaped = np.tile(UC_NQC_VSM_train, (1, code_NQC_VSM_train.shape[0]))
code_NQC_VSM_train_reshaped = np.tile(code_NQC_VSM_train, (1, UC_NQC_VSM_train.shape[0]))

UC_NQC_BM25_train_reshaped = np.tile(UC_NQC_BM25_train, (1, code_NQC_BM25_train.shape[0]))
code_NQC_BM25_train_reshaped = np.tile(code_NQC_BM25_train, (1, UC_NQC_BM25_train.shape[0]))

UC_NQC_JM_train_reshaped = np.tile(UC_NQC_JM_train, (1, code_NQC_JM_train.shape[0]))
code_NQC_JM_train_reshaped = np.tile(code_NQC_JM_train, (1, UC_NQC_JM_train.shape[0]))

UC_NQC_DP_train_reshaped = np.tile(UC_NQC_DP_train, (1, code_NQC_DP_train.shape[0]))
code_NQC_DP_train_reshaped = np.tile(code_NQC_DP_train, (1, UC_NQC_DP_train.shape[0]))



In [34]:
num_terms_code_train_reshaped = np.tile(num_terms_code_train, (num_terms_UC_train.shape[0],1))
num_terms_UC_train_reshaped = np.tile(num_terms_UC_train, (num_terms_code_train.shape[0],1))

num_unique_terms_UC_train_reshaped = np.tile(num_unique_terms_UC_train, (num_unique_terms_code_train.shape[0],1))
num_unique_terms_code_train_reshaped = np.tile(num_unique_terms_code_train, (num_unique_terms_UC_train.shape[0],1))


In [39]:
feature_matrix_train = np.stack(( cosine_similarity_UC_train, cosine_similarity_CC_train,LSA_similarities_UC_train,LSA_similarities_CC_train,LDA_similarities_UC_train,LDA_similarities_CC_train,JS_UC_train,JS_CC_train,BM25_UC_train.T,BM25_CC_train,JM_UC_train.T,JM_CC_train,DP_UC_train.T,DP_CC_train,
                           
   avg_idf_uc_train_reshaped,avg_idf_code_train_reshaped.T,max_idf_uc_train_reshaped,max_idf_code_train_reshaped.T,
    dev_idf_uc_train_reshaped,dev_idf_code_train_reshaped.T,avg_ictf_uc_train_reshaped,avg_ictf_code_train_reshaped.T,max_ictf_uc_train_reshaped,max_ictf_code_train_reshaped.T,dev_ictf_uc_train_reshaped,dev_ictf_code_train_reshaped.T,avg_entropy_uc_train_reshaped,avg_entropy_code_train_reshaped.T,max_entropy_uc_train_reshaped,max_entropy_code_train_reshaped.T,med_entropy_uc_train_reshaped,med_entropy_code_train_reshaped.T,dev_entropy_uc_train_reshaped,dev_entropy_code_train_reshaped.T,avg_variance_uc_train_reshaped,avg_variance_code_train_reshaped.T,max_variance_uc_train_reshaped,max_variance_code_train_reshaped.T,sum_variance_uc_train_reshaped,sum_variance_code_train_reshaped.T,avg_scq_uc_train_reshaped,avg_scq_code_train_reshaped.T,max_scq_uc_train_reshaped,max_scq_code_train_reshaped.T,sum_sqc_uc_train_reshaped,sum_sqc_code_train_reshaped.T,avg_pmi_uc_train_reshaped,avg_pmi_code_train_reshaped.T,max_pmi_uc_train_reshaped,max_pmi_code_train_reshaped.T,qs_uc_train_reshaped,qs_code_train_reshaped.T,UC_SCS_train_reshaped,CC_SCS_train_reshaped.T,UC_CoherenceScore_train_reshaped,CC_CoherenceScore_train_reshaped.T,

    UC_FRC_DP_train_reshaped,code_FRC_DP_train_reshaped.T,UC_FRC_JM_train_reshaped,code_FRC_JM_train_reshaped.T,UC_FRC_BM25_train_reshaped,code_FRC_BM25_train_reshaped.T,UC_FRC_JS_train_reshaped,code_FRC_JS_train_reshaped.T,UC_FRC_VSM_train_reshaped,code_FRC_VSM_train_reshaped.T

    ,UC_RS_DP_train_reshaped,code_RS_DP_train_reshaped.T,UC_RS_JM_train_reshaped,code_RS_JM_train_reshaped.T,UC_RS_BM25_train_reshaped,code_RS_BM25_train_reshaped.T,UC_RS_JS_train_reshaped,code_RS_JS_train_reshaped.T,UC_RS_VSM_train_reshaped,code_RS_VSM_train_reshaped.T,

    UC_CT_DP_train_reshaped,code_CT_DP_train_reshaped.T,UC_CT_JM_train_reshaped,
    code_CT_JM_train_reshaped.T,UC_CT_BM25_train_reshaped,code_CT_BM25_train_reshaped.T,UC_CT_JensenShannon_train_reshaped,code_CT_JensenShannon_train_reshaped.T,UC_CT_VSM_train_reshaped,code_CT_VSM_train_reshaped.T,

    UC_SAC_JM_train_reshaped,code_SAC_JM_train_reshaped.T,
    UC_SAC_BM25_train_reshaped,code_SAC_BM25_train_reshaped.T,UC_SAC_JS_train_reshaped,code_SAC_JS_train_reshaped.T,
    UC_SAC_VSM_train_reshaped,code_SAC_VSM_train_reshaped.T,UC_SAC_DP_train_reshaped,code_SAC_JS_train_reshaped.T,

    UC_WIG_score_DP_train_reshaped,code_WIG_score_DP_train_reshaped.T,UC_WIG_score_JM_train_reshaped,
    code_WIG_score_JM_train_reshaped.T,UC_WIG_score_BM25_train_reshaped,code_WIG_score_BM25_train_reshaped.T,UC_WIG_score_JensenShannon_train_reshaped,
    code_WIG_score_JensenShannon_train_reshaped.T,UC_WIG_score_VSM_train_reshaped,
    code_WIG_score_VSM_train_reshaped.T,

    UC_NQC_DP_train_reshaped,code_NQC_DP_train_reshaped.T,UC_NQC_JM_train_reshaped,code_NQC_JM_train_reshaped.T,UC_NQC_BM25_train_reshaped,code_NQC_BM25_train_reshaped.T,UC_NQC_VSM_train_reshaped,code_NQC_VSM_train_reshaped.T,UC_NQC_JensenShannon_train_reshaped,code_NQC_JensenShannon_train_reshaped.T,

    num_terms_UC_train_reshaped.T,num_terms_code_train_reshaped,num_unique_terms_UC_train_reshaped.T,num_unique_terms_code_train_reshaped,num_overlapping_terms_train),axis=2)

print(feature_matrix_train.shape)

(2398, 1031, 121)


In [40]:
avg_idf_uc_test = np.array(avg_idf_uc_test)
avg_idf_code_test = np.array(avg_idf_code_test)
max_idf_uc_test = np.array(max_idf_uc_test)
max_idf_code_test = np.array(max_idf_code_test)
dev_idf_uc_test = np.array(dev_idf_uc_test)
dev_idf_code_test = np.array(dev_idf_code_test)
avg_ictf_uc_test = np.array(avg_ictf_uc_test)
avg_ictf_code_test = np.array(avg_ictf_code_test)
max_ictf_uc_test = np.array(max_ictf_uc_test)
max_ictf_code_test = np.array(max_ictf_code_test)
dev_ictf_uc_test = np.array(dev_ictf_uc_test)
dev_ictf_code_test = np.array(dev_ictf_code_test)
avg_entropy_uc_test = np.array(avg_entropy_uc_test)
avg_entropy_code_test = np.array(avg_entropy_code_test)
max_entropy_uc_test = np.array(max_entropy_uc_test)
max_entropy_code_test = np.array(max_entropy_code_test)
med_entropy_uc_test = np.array(med_entropy_uc_test)
med_entropy_code_test = np.array(med_entropy_code_test)
dev_entropy_uc_test = np.array(dev_entropy_uc_test)
dev_entropy_code_test = np.array(dev_entropy_code_test)
avg_variance_uc_test = np.array(avg_variance_uc_test)
avg_variance_code_test = np.array(avg_variance_code_test)
max_variance_uc_test = np.array(max_variance_uc_test)
max_variance_code_test = np.array(max_variance_code_test)
sum_variance_uc_test = np.array(sum_variance_uc_test)
sum_variance_code_test = np.array(sum_variance_code_test)
avg_scq_uc_test = np.array(avg_scq_uc_test)
avg_scq_code_test = np.array(avg_scq_code_test)
max_scq_uc_test = np.array(max_scq_uc_test)
max_scq_code_test = np.array(max_scq_code_test)
sum_sqc_uc_test = np.array(sum_sqc_uc_test)
sum_sqc_code_test = np.array(sum_sqc_code_test)
avg_pmi_uc_test = np.array(avg_pmi_uc_test)
avg_pmi_code_test = np.array(avg_pmi_code_test)
max_pmi_uc_test = np.array(max_pmi_uc_test)
max_pmi_code_test = np.array(max_pmi_code_test)
qs_uc_test = np.array(qs_uc_test)
qs_code_test = np.array(qs_code_test)
UC_SCS_test = np.array(UC_SCS_test)
CC_SCS_test = np.array(CC_SCS_test)
UC_CoherenceScore_test = np.array(UC_CoherenceScore_test)
CC_CoherenceScore_test = np.array(CC_CoherenceScore_test)

In [41]:
avg_idf_uc_test_reshaped = np.tile(avg_idf_uc_test, (1, avg_idf_code_test.shape[0]))
avg_idf_code_test_reshaped = np.tile(avg_idf_code_test, (1, avg_idf_uc_test.shape[0]))

max_idf_uc_test_reshaped = np.tile(max_idf_uc_test, (1, max_idf_code_test.shape[0]))
max_idf_code_test_reshaped = np.tile(max_idf_code_test, (1, max_idf_uc_test.shape[0]))

dev_idf_uc_test_reshaped = np.tile(dev_idf_uc_test, (1, dev_idf_code_test.shape[0]))
dev_idf_code_test_reshaped = np.tile(dev_idf_code_test, (1, dev_idf_uc_test.shape[0]))

avg_ictf_uc_test_reshaped = np.tile(avg_ictf_uc_test, (1, avg_ictf_code_test.shape[0]))
avg_ictf_code_test_reshaped = np.tile(avg_ictf_code_test, (1, avg_ictf_uc_test.shape[0]))

max_ictf_uc_test_reshaped = np.tile(max_ictf_uc_test, (1, max_ictf_code_test.shape[0]))
max_ictf_code_test_reshaped = np.tile(max_ictf_code_test, (1, max_ictf_uc_test.shape[0]))

dev_ictf_uc_test_reshaped = np.tile(dev_ictf_uc_test, (1, dev_ictf_code_test.shape[0]))
dev_ictf_code_test_reshaped = np.tile(dev_ictf_code_test, (1, dev_ictf_uc_test.shape[0]))

avg_entropy_uc_test_reshaped = np.tile(avg_entropy_uc_test, (1, avg_entropy_code_test.shape[0]))
avg_entropy_code_test_reshaped = np.tile(avg_entropy_code_test, (1, avg_entropy_uc_test.shape[0]))

max_entropy_uc_test_reshaped = np.tile(max_entropy_uc_test, (1, max_entropy_code_test.shape[0]))
max_entropy_code_test_reshaped = np.tile(max_entropy_code_test, (1, max_entropy_uc_test.shape[0]))

med_entropy_uc_test_reshaped = np.tile(med_entropy_uc_test, (1, med_entropy_code_test.shape[0]))
med_entropy_code_test_reshaped = np.tile(med_entropy_code_test, (1, med_entropy_uc_test.shape[0]))

dev_entropy_uc_test_reshaped = np.tile(dev_entropy_uc_test, (1, dev_entropy_code_test.shape[0]))
dev_entropy_code_test_reshaped = np.tile(dev_entropy_code_test, (1, dev_entropy_uc_test.shape[0]))

avg_variance_uc_test_reshaped = np.tile(avg_variance_uc_test, (1, avg_variance_code_test.shape[0]))
avg_variance_code_test_reshaped = np.tile(avg_variance_code_test, (1, avg_variance_uc_test.shape[0]))

max_variance_uc_test_reshaped = np.tile(max_variance_uc_test, (1, max_variance_code_test.shape[0]))
max_variance_code_test_reshaped = np.tile(max_variance_code_test, (1, max_variance_uc_test.shape[0]))

sum_variance_uc_test_reshaped = np.tile(sum_variance_uc_test, (1, sum_variance_code_test.shape[0]))
sum_variance_code_test_reshaped = np.tile(sum_variance_code_test, (1, sum_variance_uc_test.shape[0]))

avg_scq_uc_test_reshaped = np.tile(avg_scq_uc_test, (1, avg_scq_code_test.shape[0]))
avg_scq_code_test_reshaped = np.tile(avg_scq_code_test, (1, avg_scq_uc_test.shape[0]))

max_scq_uc_test_reshaped = np.tile(max_scq_uc_test, (1, max_scq_code_test.shape[0]))
max_scq_code_test_reshaped = np.tile(max_scq_code_test, (1, max_scq_uc_test.shape[0]))

sum_sqc_uc_test_reshaped = np.tile(sum_sqc_uc_test, (1, sum_sqc_code_test.shape[0]))
sum_sqc_code_test_reshaped = np.tile(sum_sqc_code_test, (1, sum_sqc_uc_test.shape[0]))

avg_pmi_uc_test_reshaped = np.tile(avg_pmi_uc_test, (1, avg_pmi_code_test.shape[0]))
avg_pmi_code_test_reshaped = np.tile(avg_pmi_code_test, (1, avg_pmi_uc_test.shape[0]))

max_pmi_uc_test_reshaped = np.tile(max_pmi_uc_test, (1, max_pmi_code_test.shape[0]))
max_pmi_code_test_reshaped = np.tile(max_pmi_code_test, (1, max_pmi_uc_test.shape[0]))

qs_uc_test_reshaped = np.tile(qs_uc_test, (1, qs_code_test.shape[0]))
qs_code_test_reshaped = np.tile(qs_code_test, (1, qs_uc_test.shape[0]))

UC_SCS_test_reshaped = np.tile(UC_SCS_test, (1, CC_SCS_test.shape[0]))
CC_SCS_test_reshaped = np.tile(CC_SCS_test, (1, UC_SCS_test.shape[0]))

UC_CoherenceScore_test_reshaped = np.tile(UC_CoherenceScore_test, (1, CC_CoherenceScore_test.shape[0]))
CC_CoherenceScore_test_reshaped = np.tile(CC_CoherenceScore_test, (1, UC_CoherenceScore_test.shape[0]))


In [42]:
UC_RS_JS_test_reshaped = np.tile(UC_RS_JS_test, (1, code_RS_JS_test.shape[0]))
code_RS_JS_test_reshaped = np.tile(code_RS_JS_test, (1, UC_RS_JS_test.shape[0]))

UC_FRC_JS_test_reshaped = np.tile(UC_FRC_JS_test, (1, code_FRC_JS_test.shape[0]))
code_FRC_JS_test_reshaped = np.tile(code_FRC_JS_test, (1, UC_FRC_JS_test.shape[0]))

UC_RS_VSM_test_reshaped = np.tile(UC_RS_VSM_test, (1, code_RS_VSM_test.shape[0]))
code_RS_VSM_test_reshaped = np.tile(code_RS_VSM_test, (1, UC_RS_VSM_test.shape[0]))

UC_FRC_VSM_test_reshaped = np.tile(UC_FRC_VSM_test, (1, code_FRC_VSM_test.shape[0]))
code_FRC_VSM_test_reshaped = np.tile(code_FRC_VSM_test, (1, UC_FRC_VSM_test.shape[0]))

UC_RS_BM25_test_reshaped = np.tile(UC_RS_BM25_test, (1, code_RS_BM25_test.shape[0]))
code_RS_BM25_test_reshaped = np.tile(code_RS_BM25_test, (1, UC_RS_BM25_test.shape[0]))

UC_FRC_BM25_test_reshaped = np.tile(UC_FRC_BM25_test, (1, code_FRC_BM25_test.shape[0]))
code_FRC_BM25_test_reshaped = np.tile(code_FRC_BM25_test, (1, UC_FRC_BM25_test.shape[0]))

UC_RS_JM_test_reshaped = np.tile(UC_RS_JM_test, (1, code_RS_JM_test.shape[0]))
code_RS_JM_test_reshaped = np.tile(code_RS_JM_test, (1, UC_RS_JM_test.shape[0]))

UC_FRC_JM_test_reshaped = np.tile(UC_FRC_JM_test, (1, code_FRC_JM_test.shape[0]))
code_FRC_JM_test_reshaped = np.tile(code_FRC_JM_test, (1, UC_FRC_JM_test.shape[0]))

UC_RS_DP_test_reshaped = np.tile(UC_RS_DP_test, (1, code_RS_DP_test.shape[0]))
code_RS_DP_test_reshaped = np.tile(code_RS_DP_test, (1, UC_RS_DP_test.shape[0]))

UC_FRC_DP_test_reshaped = np.tile(UC_FRC_DP_test, (1, code_FRC_DP_test.shape[0]))
code_FRC_DP_test_reshaped = np.tile(code_FRC_DP_test, (1, UC_FRC_DP_test.shape[0]))


In [43]:
UC_CT_JensenShannon_test_reshaped = np.tile(UC_CT_JensenShannon_test, (1, code_CT_JensenShannon_test.shape[0]))
code_CT_JensenShannon_test_reshaped = np.tile(code_CT_JensenShannon_test, (1, UC_CT_JensenShannon_test.shape[0]))

UC_CT_VSM_test_reshaped = np.tile(UC_CT_VSM_test, (1, code_CT_VSM_test.shape[0]))
code_CT_VSM_test_reshaped = np.tile(code_CT_VSM_test, (1, UC_CT_VSM_test.shape[0]))

UC_CT_BM25_test_reshaped = np.tile(UC_CT_BM25_test, (1, code_CT_BM25_test.shape[0]))
code_CT_BM25_test_reshaped = np.tile(code_CT_BM25_test, (1, UC_CT_BM25_test.shape[0]))

UC_CT_JM_test_reshaped = np.tile(UC_CT_JM_test, (1, code_CT_JM_test.shape[0]))
code_CT_JM_test_reshaped = np.tile(code_CT_JM_test, (1, UC_CT_JM_test.shape[0]))

UC_CT_DP_test_reshaped = np.tile(UC_CT_DP_test, (1, code_CT_DP_test.shape[0]))
code_CT_DP_test_reshaped = np.tile(code_CT_DP_test, (1, UC_CT_DP_test.shape[0]))


In [44]:
UC_SAC_JS_test_reshaped = np.tile(UC_SAC_JS_test, (1, code_SAC_JS_test.shape[0]))
code_SAC_JS_test_reshaped = np.tile(code_SAC_JS_test, (1, UC_SAC_JS_test.shape[0]))

UC_SAC_VSM_test_reshaped = np.tile(UC_SAC_VSM_test, (1, code_SAC_VSM_test.shape[0]))
code_SAC_VSM_test_reshaped = np.tile(code_SAC_VSM_test, (1, UC_SAC_VSM_test.shape[0]))

UC_SAC_BM25_test_reshaped = np.tile(UC_SAC_BM25_test, (1, code_SAC_BM25_test.shape[0]))
code_SAC_BM25_test_reshaped = np.tile(code_SAC_BM25_test, (1, UC_SAC_BM25_test.shape[0]))

UC_SAC_JM_test_reshaped = np.tile(UC_SAC_JM_test, (1, code_SAC_JM_test.shape[0]))
code_SAC_JM_test_reshaped = np.tile(code_SAC_JM_test, (1, UC_SAC_JM_test.shape[0]))

UC_SAC_DP_test_reshaped = np.tile(UC_SAC_DP_test, (1, code_SAC_DP_test.shape[0]))
code_SAC_DP_test_reshaped = np.tile(code_SAC_DP_test, (1, UC_SAC_DP_test.shape[0]))

In [45]:
UC_WIG_score_JensenShannon_test_reshaped = np.tile(UC_WIG_score_JensenShannon_test, (1, code_WIG_score_JensenShannon_test.shape[0]))
code_WIG_score_JensenShannon_test_reshaped = np.tile(code_WIG_score_JensenShannon_test, (1, UC_WIG_score_JensenShannon_test.shape[0]))

UC_WIG_score_VSM_test_reshaped = np.tile(UC_WIG_score_VSM_test, (1, code_WIG_score_VSM_test.shape[0]))
code_WIG_score_VSM_test_reshaped = np.tile(code_WIG_score_VSM_test, (1, UC_WIG_score_VSM_test.shape[0]))

UC_WIG_score_BM25_test_reshaped = np.tile(UC_WIG_score_BM25_test, (1, code_WIG_score_BM25_test.shape[0]))
code_WIG_score_BM25_test_reshaped = np.tile(code_WIG_score_BM25_test, (1, UC_WIG_score_BM25_test.shape[0]))

UC_WIG_score_JM_test_reshaped = np.tile(UC_WIG_score_JM_test, (1, code_WIG_score_JM_test.shape[0]))
code_WIG_score_JM_test_reshaped = np.tile(code_WIG_score_JM_test, (1, UC_WIG_score_JM_test.shape[0]))

UC_WIG_score_DP_test_reshaped = np.tile(UC_WIG_score_DP_test, (1, code_WIG_score_DP_test.shape[0]))
code_WIG_score_DP_test_reshaped = np.tile(code_WIG_score_DP_test, (1, UC_WIG_score_DP_test.shape[0]))


In [46]:
UC_NQC_JensenShannon_test_reshaped = np.tile(UC_NQC_JensenShannon_test, (1, code_NQC_JensenShannon_test.shape[0]))
code_NQC_JensenShannon_test_reshaped = np.tile(code_NQC_JensenShannon_test, (1, UC_NQC_JensenShannon_test.shape[0]))

UC_NQC_VSM_test_reshaped = np.tile(UC_NQC_VSM_test, (1, code_NQC_VSM_test.shape[0]))
code_NQC_VSM_test_reshaped = np.tile(code_NQC_VSM_test, (1, UC_NQC_VSM_test.shape[0]))

UC_NQC_BM25_test_reshaped = np.tile(UC_NQC_BM25_test, (1, code_NQC_BM25_test.shape[0]))
code_NQC_BM25_test_reshaped = np.tile(code_NQC_BM25_test, (1, UC_NQC_BM25_test.shape[0]))

UC_NQC_JM_test_reshaped = np.tile(UC_NQC_JM_test, (1, code_NQC_JM_test.shape[0]))
code_NQC_JM_test_reshaped = np.tile(code_NQC_JM_test, (1, UC_NQC_JM_test.shape[0]))

UC_NQC_DP_test_reshaped = np.tile(UC_NQC_DP_test, (1, code_NQC_DP_test.shape[0]))
code_NQC_DP_test_reshaped = np.tile(code_NQC_DP_test, (1, UC_NQC_DP_test.shape[0]))

In [47]:
num_terms_code_test_reshaped = np.tile(num_terms_code_test, (num_terms_UC_test.shape[0], 1))
num_terms_UC_test_reshaped = np.tile(num_terms_UC_test, (num_terms_code_test.shape[0], 1))

num_unique_terms_UC_test_reshaped = np.tile(num_unique_terms_UC_test, (num_unique_terms_code_test.shape[0], 1))
num_unique_terms_code_test_reshaped = np.tile(num_unique_terms_code_test, (num_unique_terms_UC_test.shape[0], 1))


In [48]:
feature_matrix_test = np.stack([
    cosine_similarity_UC_test,cosine_similarity_CC_test,LSA_similarities_UC_test,LSA_similarities_CC_test,
    LDA_similarities_UC_test,LDA_similarities_CC_test,JS_UC_test,JS_CC_test,BM25_UC_test.T, BM25_CC_test, JM_UC_test.T, JM_CC_test, DP_UC_test.T, DP_CC_test, 

    avg_idf_uc_test_reshaped, avg_idf_code_test_reshaped.T, max_idf_uc_test_reshaped, max_idf_code_test_reshaped.T,dev_idf_uc_test_reshaped, dev_idf_code_test_reshaped.T, avg_ictf_uc_test_reshaped, avg_ictf_code_test_reshaped.T, max_ictf_uc_test_reshaped, max_ictf_code_test_reshaped.T, dev_ictf_uc_test_reshaped, dev_ictf_code_test_reshaped.T, avg_entropy_uc_test_reshaped, avg_entropy_code_test_reshaped.T, max_entropy_uc_test_reshaped, max_entropy_code_test_reshaped.T, med_entropy_uc_test_reshaped, med_entropy_code_test_reshaped.T, dev_entropy_uc_test_reshaped, dev_entropy_code_test_reshaped.T, avg_variance_uc_test_reshaped,avg_variance_code_test_reshaped.T,max_variance_uc_test_reshaped,max_variance_code_test_reshaped.T,sum_variance_uc_test_reshaped, sum_variance_code_test_reshaped.T, avg_scq_uc_test_reshaped, avg_scq_code_test_reshaped.T,  max_scq_uc_test_reshaped,  max_scq_code_test_reshaped.T,  sum_sqc_uc_test_reshaped,  sum_sqc_code_test_reshaped.T,avg_pmi_uc_test_reshaped, avg_pmi_code_test_reshaped.T,max_pmi_uc_test_reshaped,max_pmi_code_test_reshaped.T,qs_uc_test_reshaped,qs_code_test_reshaped.T,UC_SCS_test_reshaped,CC_SCS_test_reshaped.T,UC_CoherenceScore_test_reshaped,CC_CoherenceScore_test_reshaped.T,
    
    UC_FRC_DP_test_reshaped,code_FRC_DP_test_reshaped.T,UC_FRC_JM_test_reshaped,code_FRC_JM_test_reshaped.T,UC_FRC_BM25_test_reshaped,code_FRC_BM25_test_reshaped.T,UC_FRC_JS_test_reshaped,code_FRC_JS_test_reshaped.T,UC_FRC_VSM_test_reshaped,code_FRC_VSM_test_reshaped.T,
    
    UC_RS_DP_test_reshaped,code_RS_DP_test_reshaped.T,UC_RS_JM_test_reshaped,code_RS_JM_test_reshaped.T,UC_RS_BM25_test_reshaped,code_RS_BM25_test_reshaped.T,UC_RS_JS_test_reshaped,code_RS_JS_test_reshaped.T,UC_RS_VSM_test_reshaped,code_RS_VSM_test_reshaped.T,
    
    UC_CT_DP_test_reshaped,code_CT_DP_test_reshaped.T,UC_CT_JM_test_reshaped,code_CT_JM_test_reshaped.T,UC_CT_BM25_test_reshaped,code_CT_BM25_test_reshaped.T,UC_CT_JensenShannon_test_reshaped,code_CT_JensenShannon_test_reshaped.T,UC_CT_VSM_test_reshaped,code_CT_VSM_test_reshaped.T,
    
    UC_SAC_JM_test_reshaped,code_SAC_JM_test_reshaped.T,UC_SAC_BM25_test_reshaped,code_SAC_BM25_test_reshaped.T,UC_SAC_JS_test_reshaped,code_SAC_JS_test_reshaped.T,UC_SAC_VSM_test_reshaped,code_SAC_VSM_test_reshaped.T,UC_SAC_DP_test_reshaped,code_SAC_DP_test_reshaped.T
    
    ,UC_WIG_score_DP_test_reshaped,code_WIG_score_DP_test_reshaped.T,UC_WIG_score_JM_test_reshaped,code_WIG_score_JM_test_reshaped.T,UC_WIG_score_BM25_test_reshaped,code_WIG_score_BM25_test_reshaped.T,UC_WIG_score_JensenShannon_test_reshaped,code_WIG_score_JensenShannon_test_reshaped.T,UC_WIG_score_VSM_test_reshaped,code_WIG_score_VSM_test_reshaped.T,
    
    UC_NQC_DP_test_reshaped,code_NQC_DP_test_reshaped.T,UC_NQC_JM_test_reshaped,code_NQC_JM_test_reshaped.T,UC_NQC_BM25_test_reshaped,code_NQC_BM25_test_reshaped.T,UC_NQC_VSM_test_reshaped,code_NQC_VSM_test_reshaped.T,UC_NQC_JensenShannon_test_reshaped,code_NQC_JensenShannon_test_reshaped.T,
    
    num_terms_UC_test_reshaped.T,num_terms_code_test_reshaped,num_unique_terms_UC_test_reshaped.T,num_unique_terms_code_test_reshaped,num_overlapping_terms_test
], axis=2)

# Print the shape of the feature matrix
print(feature_matrix_test.shape)


(409, 174, 121)


## Feature Selection

In [49]:
# matrix shape = links * features 
# ----features------
#|
#links
#|
feature_matrix_train_reshaped = feature_matrix_train.reshape(feature_matrix_train.shape[0]*feature_matrix_train.shape[1], -1)
print(feature_matrix_train_reshaped.shape)
correlation_features_train=np.corrcoef(feature_matrix_train_reshaped,rowvar=False)
print(correlation_features_train.shape)
features_excluded_train=set()

for i in range(correlation_features_train.shape[1]):
    for j in range(i+1,correlation_features_train.shape[0]):
        if (correlation_features_train[j][i] >= 0.9 ):
            features_excluded_train.add(j)

features_links_selected_train=np.delete(feature_matrix_train_reshaped, list(features_excluded_train), axis=1) 
print(feature_matrix_train_reshaped)
print(features_links_selected_train.shape)
features_links_selected_reshaped_train = features_links_selected_train.reshape(feature_matrix_train.shape[0], feature_matrix_train.shape[1], -1)
print(features_links_selected_reshaped_train.shape)

(2472338, 121)
(121, 121)
[[7.83132530e-01 9.78696742e-01 3.04007820e-01 ... 4.40000000e+01
  4.43000000e+02 2.05338809e+00]
 [6.25122669e-01 9.16383701e-01 7.63956905e-01 ... 4.40000000e+01
  2.11000000e+02 2.74509804e+00]
 [3.12195122e-02 3.73070326e-02 9.21129503e-01 ... 4.40000000e+01
  1.78000000e+02 4.95495495e+00]
 ...
 [9.86368062e-01 9.52696729e-01 8.45029240e-01 ... 2.80000000e+01
  1.13000000e+02 1.41843972e+00]
 [3.43503937e-01 5.21573604e-01 6.54669261e-01 ... 2.80000000e+01
  1.76000000e+02 2.94117647e+00]
 [7.97989950e-01 9.34819176e-01 2.55609756e-01 ... 2.80000000e+01
  2.09000000e+02 2.10970464e+00]]
(2472338, 78)
(2398, 1031, 78)


In [50]:
# matrix shape = links * features 
# ----features------
#|
#links
#|
feature_matrix_test_reshaped = feature_matrix_test.reshape(feature_matrix_test.shape[0]*feature_matrix_test.shape[1], -1)
print(feature_matrix_test_reshaped.shape)
correlation_features_test=np.corrcoef(feature_matrix_test_reshaped,rowvar=False)
print(correlation_features_test.shape)
features_excluded_test=set()

for i in range(correlation_features_test.shape[1]):
    for j in range(i+1,correlation_features_test.shape[0]):
        if (correlation_features_test[j][i] >= 0.9 ):
            features_excluded_test.add(j)

features_links_selected_test=np.delete(feature_matrix_test_reshaped, list(features_excluded_train), axis=1) 
print(feature_matrix_test_reshaped)
print(features_links_selected_test.shape)
features_links_selected_reshaped_test = features_links_selected_test.reshape(feature_matrix_test.shape[0], feature_matrix_test.shape[1], -1)
print(features_links_selected_reshaped_test.shape)

(71166, 121)


(121, 121)
[[2.14285714e-01 4.82843137e-01 2.67857143e-01 ... 5.00000000e+01
  5.02000000e+02 3.62318841e+00]
 [4.15204678e-01 4.76658477e-01 4.65116279e-01 ... 5.00000000e+01
  2.67000000e+02 4.73186120e+00]
 [2.94117647e-01 1.64215686e-01 7.03488372e-01 ... 5.00000000e+01
  2.94000000e+02 4.65116279e+00]
 ...
 [1.00000000e+00 1.00000000e+00 8.13953488e-01 ... 2.60000000e+01
  8.90000000e+01 0.00000000e+00]
 [7.39884393e-01 6.97406340e-01 6.08187135e-01 ... 2.60000000e+01
  5.80000000e+01 2.38095238e+00]
 [8.60465116e-01 8.83663366e-01 0.00000000e+00 ... 2.60000000e+01
  1.57000000e+02 2.18579235e+00]]
(71166, 78)
(409, 174, 78)


c:\ProgramData\anaconda3\envs\cude_env2\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\ProgramData\anaconda3\envs\cude_env2\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


### Mapping Features to Labels

In [51]:
Features_train = list()
DataSet_train = pd.read_csv('Dataset/teiid_dataset/train_modified.csv')
for row in DataSet_train.index:
    index_code = int(DataSet_train.loc[row, 'CC'])
    index_UC = int(DataSet_train.loc[row, 'UC'])
    Features_train.append(features_links_selected_reshaped_train[index_UC][index_code])


In [52]:
Features_test = list()
DataSet_test = pd.read_csv('Dataset/teiid_dataset/test_modified.csv')
for row in DataSet_test.index:
    index_code = int(DataSet_test.loc[row, 'CC'])
    index_UC = int(DataSet_test.loc[row, 'UC'])
    Features_test.append(features_links_selected_reshaped_test[index_UC][index_code])


In [ ]:
# print(len(Features[]))
# print(len(DataSet['Labels'].to_list()))

## Data Imbalancing

In [53]:
from imblearn.over_sampling import BorderlineSMOTE 
smote = BorderlineSMOTE(random_state=42)
Features_SMOTE_train, Labels_SMOTE_train = smote.fit_resample(Features_train, DataSet_train['Labels'].to_list())

## Model

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# train_features, train_labels = train_test_split(Features_SMOTE_train, Labels_SMOTE_train, test_size = 0, random_state = 42)
model_random_forest = RandomForestRegressor(n_estimators = 10, random_state = 42, verbose=2, n_jobs=4)
model_random_forest.fit(Features_SMOTE_train, Labels_SMOTE_train)


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 4 of 10building tree 1 of 10
building tree 2 of 10
building tree 3 of 10

building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed: 131.4min finished


RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=42, verbose=2)

In [55]:
import joblib
joblib.dump(model_random_forest, 'Dataset/teiid_dataset/teiid_model_new.pkl')

['Dataset/teiid_dataset/teiid_model_new.pkl']

In [56]:
test_labels = DataSet_test['Labels'].to_list()
predictions = model_random_forest.predict(Features_test)
errors = abs(predictions - test_labels)
print('mean Absolute Error:', round(np.mean(errors), 2) )

acc=(test_labels==predictions).sum()
print(acc/len(test_labels))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.3s finished


mean Absolute Error: 0.15
0.4593766686338982


Loading Model and Testing

In [57]:
from joblib import load

# Load the model
model = load('Dataset/teiid_dataset/teiid_model_new.pkl')

# Assuming you have test data in variables X_test and y_test
predictions = model.predict(Features_test)

# Calculate the accuracy of the model on the test data
test_labels = DataSet_test['Labels'].to_list()
errors = abs(predictions - test_labels)
print('mean Absolute Error:', round(np.mean(errors), 2) )

acc=(test_labels==predictions).sum()
print('Accuracy:',acc/len(test_labels))

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.5s finished


mean Absolute Error: 0.15
Accuracy: 0.4593766686338982


In [58]:
print("test labels: ", test_labels)
print("predictions: ")
for prediction in predictions:
    print(prediction)

test labels:  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# from joblib import dump

# # Assuming you have trained a model named 'model'
# # You can save it using dump
# dump(model_random_forest, 'RandomForst_121Features_1st_trial.joblib')


In [ ]:
import csv
with open('./dataset/answerSet.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    valid_links_labels = []
    for row in reader:
        temp=row[0].split(",")
        #match = re.search(rprint('UC(\d+)\.txt', temp[0])
        valid_links_labels.append((temp[0],temp[1]))
        # if (valid_links_labels.get(temp[1])==None):
        # valid_links_labels[temp[1]]=[int(match.group(1))]
        # else:
        #     valid_links_labels[temp[1]].append(int(match.group(1)))

# file_names = list(valid_links_labels.keys())
# file_names.sort()
# valid_links_labels_sorted = {i: valid_links_labels[i] for i in file_names}
            
# valid_links_labels_sorted)
# len(valid_links_labels_sorted.keys()))
print(valid_links_labels)